In [ ]:
from google.cloud import bigquery

In [ ]:
client=bigquery.Client()

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
%reload_ext google.cloud.bigquery

In [ ]:
#######################################
print('Setting everything up...')
#######################################

import warnings
warnings.filterwarnings('ignore')
import pandas_gbq 
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.lines import Line2D

import matplotlib.ticker as ticker
import matplotlib.cm as cm
import matplotlib as mpl

import matplotlib.pyplot as plt
%matplotlib inline



import os
import sys
from datetime import datetime
from datetime import date
from datetime import time
from datetime import timedelta
import time

DATASET = ''

plt.style.use('ggplot')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

from IPython.display import HTML as html_print

def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)


print('done.')


In [ ]:
dic={'src_hpo_id':["trans_am_essentia","saou_ummc","seec_miami","seec_morehouse","seec_emory","uamc_banner","pitt","nyc_cu","ipmc_uic","trans_am_spectrum","tach_hfhs","nec_bmc","cpmc_uci","nec_phs","nyc_cornell","ipmc_nu","nyc_hh","ipmc_uchicago","aouw_mcri","syhc","cpmc_ceders","seec_ufl","saou_uab","trans_am_baylor","cpmc_ucsd","ecchc","chci","aouw_uwh","cpmc_usc","hrhc","ipmc_northshore","chs","cpmc_ucsf","jhchc","aouw_mcw","cpmc_ucd","ipmc_rush"],
    'HPO':["Essentia Health Superior Clinic","University of Mississippi","SouthEast Enrollment Center Miami","SouthEast Enrollment Center Morehouse","SouthEast Enrollment Center Emory","Banner Health","University of Pittsburgh","Columbia University Medical Center","University of Illinois Chicago","Spectrum Health","Henry Ford Health System","Boston Medical Center","UC Irvine","Partners HealthCare","Weill Cornell Medical Center","Northwestern Memorial Hospital","Harlem Hospital","University of Chicago","Marshfield Clinic","San Ysidro Health Center","Cedars-Sinai","University of Florida","University of Alabama at Birmingham","Baylor","UC San Diego","Eau Claire Cooperative Health Center","Community Health Center, Inc.","UW Health (University of Wisconsin Madison)","University of Southern California","HRHCare","NorthShore University Health System","Cherokee Health Systems","UC San Francisco","Jackson-Hinds CHC","Medical College of Wisconsin","UC Davis","Rush University"]}


site_df=pd.DataFrame(data=dic)
site_df

In [ ]:
measurement_codes=(3015182,3019897,3020416,3023314,3000963,3000905,3012030,3009744,3023599,3002736,3043111,3024929,3004809,
                   3022096,3025159,3006504,3009542,3000963,3002030,3035941,3003338,3024731,3002179,3019069,3017181,3018010,
                   3007461,3003214,3021440,3012764,3026361,3027945,3008757,3010813,3006923,3024561,3020509,3035995,3013721,
                   3006906,3016723,3045716,3016293,3015632,3014576,3023103,3019550,3027970,3029829,3030354,3004501,3020630,
                   3024128,3013682,3018311,3053283,3049187,3006906,3016723,3045716,3016293,3015632,3014576,3023103,3019550,
                   3029829,3030354,3004501,3013682,3018311,3053283,3049187,3004249,3012888,3027598,3036277,3019204,3020891,
                   3013762,3027018,3024171,3019900,3027114,3049873,3007070,3023602,3034482,3053286,3001308,3028288,3028437,
                   3035009,3035899,3038988,42870529,3009966,3053341,3002109,3045001,3007352,3009596,3022487,40768809,3011163
                   ,3016087,3007943,3019038,3013678,3022038,3022192,3025839,3027997,42868692)

# Integration of Units for Select Measurements:


In [ ]:
unit_standard_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS unit_total_row
            FROM
               `{}.unioned_ehr_measurement` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS t2
            ON
                t1.measurement_id=t2.measurement_id
            WHERE measurement_concept_id in {}
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS unit_well_defined_row
            FROM
               `{}.unioned_ehr_measurement` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS t2
            ON
                t1.measurement_id=t2.measurement_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.unit_concept_id
            WHERE 
                 t3.standard_concept="S"
                 and
                 measurement_concept_id in {}
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        unit_well_defined_row,
        unit_total_row,
        round(100*(unit_well_defined_row/unit_total_row),1) as unit_success_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,measurement_codes,DATASET,DATASET,DATASET,measurement_codes,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
unit_standard_df.shape

In [ ]:
unit_standard_df=unit_standard_df.fillna(0)

In [ ]:
unit_standard_df

In [ ]:
unit_standard_df.to_csv("data\q7_Integration_Units_Select_Measurements.csv")

#  Integration of Routes for Select Drugs:


In [ ]:
antibiotics=(40182266,40131823,35604887,948637,19117654,40240571,40159304,40232630,19075667,19117134,19010293,1729746,40180927,19074842,
45892578,19001211,956653,19111311,19006597,19023508,19133883,35603136,19006346,46287369,40170801,40229284,1769597,40234552,
19006462,19117750,19081972,1777806,19112888,40021623,40182269,46275708,904516,42901613,19063139,19006833,40178235,46275105,
19006725,40145364,19068186,40092540,19070252,19136024,19111992,40182615,19111047,1713960,40084555,19118038,19107412,19133908,
1713638,40153795,35604118,40163182,19009140,1713985,19111279,40173660,40158037,19086759,40069479,19001223,956661,46275712,
19112058,40040326,19027099,19030031,1786617,19111310,35605613,19073188,19133911,1734206,40137641,19103150,1768911,19128195,
19058200,19079750,1724695,45892715,19030256,19062826,19031595,19072054,19077018,1749008,19035319,19107207,42900906,35606370,
40238954,35606164,19112962,40229699,42707538,40170271,19006320,46234412,19107830,40092488,19113235,40170262,45775452,
40039205,46275525,19105765,45775715,40169666,19087038,40009467,42902600,40077149,19116606,19116937,35606420,19070257,
19070254,19107870,19020463,19106638,19133324,1836191,40077150,46287349,40182265,998001,19133010,19006390,19133880,19027100,
19006425,19082703,19097839,35606369,1746145,19006469,19105431,40088434,1708158,19074820,19108673,40064152,40009470,
19129363,19107873,19117922,1769565,19048475,19131147,40032461,1777284,19115408,19129191,19074845,19045282,40182616,
19117136,40173656,44818082,920340,40067094,19026200,42902477,19078894,1796435,40080970,19101402,19111314,19117924,
19006594,19106675,19086047,46234348,19122455,19133323,19035325,19106641,19104509,40173667,40009211,1705708,1796454,
19129642,19006381,1708960,19117823,19010178,19075037,40227700,19091820,1734134,19117791,19099312,19105304,19054962,
19128419,1713698,40021249,45775871,19133009,19123645,19081735,19010506,19108240,1702397,40084557,19001227,40170391,
40152865,46287358,40131825,40127804,19038924,1836456,19045288,19074977,40169545,40229274,19105251,19006514,19061664,
40064526,19010503,40238952,19115239,19005837,19020465,19118700,19133889,19102889,40079526,40229258,19085025,19006386,
19084153,1721549,19010504,19006411,19051271,19027097,19044423,19136429,42707537,46287329,19024445,1773410,40009218,
19103457,40229271,19105824,19071449,19082794,35606001,44814368,1777286,1768880,40024441,19006470,19060192,42874172,
40170793,19047895,19120394,40170263,19082979,19125921,1742257,19061770,997933,40147961,1724736,19061916,40232616,
40024033,46287356,19063885,1836394,19105321,19032382,43525738,19023510,46276322,46275433,40058963,40024407,19047898,
19098932,19073183,40009674,19126519,40145335,40236525,19115445,40032458,19048476,19030533,19074902,19133003,19136022,
40232578,19106661,19092321,19128708,35604161,40009202,19116071,42901625,19070251,19102382,35605997,19105291,40040318,
40141393,1797555,40067401,1705737,19006384,1776784,40040322,19099644,19047614,19015464,1738396,19030254,19016767,
19109533,40064525,19112045,19001904,40081376,40112887,40024037,40182282,920333,19085094,19041157,1836243,
42902744,19062538,19081059,19113370,19110530,19106635,40059360,40095873,19085329,1742434,42901626,19100344,
40182277,40071656,19135630,40063023,19116939,19104511,19001343,1768905,904451,19027070,1768963,40032460,19123607,
40009679,40169549,19121045,19070434,19061430,19112043,19129412,19126165,1728604,40230267,40080969,19116623,40063081,19006428,19060191,46275713,19109221,19133856,19048477,35604794,45892603,40232613,1729981,19037690,40040612,19070702,19111986,19136493,40173950,19054970,40008344,46275526,40024075,40052002,19133899,19035327,19117793,42799246,19131485,19090782,19006042,35606006,46275444,40229702,40238689,19063161,19084437,40011362,40064518,19133863,42706989,19085269,1717481,40020957,19047730,1734209,19113952,46287322,19086781,1836393,40010982,19087010,19083574,19006687,19137365,1769563,19105333,1729842,19006596,19136424,19129099,19064248,19096145,46287321,19005834,1743223,19044422,1713701,40182276,19133020,19120083,1836242,19135293,19006724,40121177,19006417,1796476,19060618,46287383,19010210,1790700,19111320,19060619,40084920,998000,19045255,40032459,1786746,19031745,19100278,19019420,19105303,19085351,35606018,42902201,19080620,40081003,19116936,19001226,1747204,40170267,19030195,1701675,19110815,1724696,19056553,19074879,40081028,40027292,19106097,19135631,19039193,19083542,40169388,46276324,19117788,46275520,19133868,40063088,40063087,19006684,1759879,1747008,46276323,19007701,40161024,19001284,40232623,19133016,19125228,19123643,19113545,40009680,19015466,19121679,40000286,40057579,40024408,1728545,40065416,19061823,19113076,19001225,19117109,1776721,19107620,19078399,19010575,40040319,40009670,1750562,40136149,40016517,19091813,19047471,19128749,40098940,19111321,19133912,19133023,19048245,19103730,45776147,40149224,19048406,40028351,43011431,40092164,40108192,19117132,40032671,1742287,40163185,40092579,40223860,46287339,19115705,19134468,46276321,19136494,45774462,40058965,19034884,1796473,40143339,40057233,35604798,19070258,19087059,19063846,40161025,40029794,46275516,40224843,19113556,40009224,35604888,19106411,46287370,19016793,1729751,19089971,19111792,42903095,35603620,19103401,40229272,19058363,45775718,19006686,1713545,40171394,19115413,1774470,40168171,19091812,40092559,40081395,19113555,46287313,1836193,19018845,19016371,19098000,40232605,19133327,40009212,43011881,42902203,40029809,40228858,19085833,40182271,42901628,19113817,44818400,19070972,46275108,19117036,19116092,35604160,19006427,19051345,19133906,40020647,997892,19036545,40000267,40145354,19046397,1734108,19010502,19072257,40071658,40064155,43525589,19133892,19071689,19103032,990115,35604626,1714527,40101871,19082513,19006342,1777254,40102429,19133923,46275434,19089717,1702933,40071756,40232625,46287307,40096062,1746114,1713633,40229277,19079771,19107880,19098205,42899811,19105324,19009353,19050774,1740546,40241902,46287350,19133896,19025857,40058687,1728575,40220674,1728943,1728611,40232395,40169384,42706329,46275443,46287360,19006341,40229261,19133012,44784833,1713930,19030507,1836453,19069006,1713412,45776342,19108779,19001253,19063165,40170255,19111054,19106463,19111023,35606374,46276320,35605656,40163178,19054940,40009502,19133011,1759847,19123186,40170266,1749060,19074980,19109217,1713418,40009191,40092561,40008461,19039123,40127531,19048604,40229264,40009226,19107973,19070680,40169226,1716904,19116078,19106552,40021309,19133014,19018154,19102336,19023509,19058221,35605657,40057469,19027094,19105555,1705674,19079772,19112418,1713520,19117399,46287345,19016794,40024749,19091637,19127878,19021249,19105349,956659,19123649,40087396,35605336,19116060,40008343,1713336,19027679,40238957,43011781,40147350,19088919,1769539,40161023,19129060,40145363,19122454,40067086,19030038,40169225,1729791,42900911,40224848,19064247,1713632,40063082,40181189,40170256,1708161,19121616,19041155,40169546,19113816,19030476,19006344,19045289,19125247,920339,19130200,1776761,19027712,19116074,19031747,19116090,1728416,19092922,19028288,40170797,46275704,19106677,40239201,35606366,40146077,19115096,40163195,19040651,19123595,46275109,19105302,40023776,19118089,40096063,19083545,19094614,40056694,40000289,19100236,19117677,19006383,19121678,19115415,19038869,19047897,1738367,19103531,40148072,19003646,40064523,1713983,904558,19133954,19031815,40087792,1716903,1712572,46287351,19006466,19023678,19102072,46234347,19096017,40229278,40009503,40173661,904515,1724668,19133881,40081781,19116091,19081057,19001251,40024355,19084260,19050750,19011481,1768934,45774460,1713472,19114002,40238955,19105753,19032383,19127617,40148071,19029074,40009478,35605339,1717507,986864,19079657,19129337,40123619,19079893,46275512,19115225,1713473,1786653,40091409,19006412,46275878,990087,19117403,40009223,1796474,19006832,19115067,19123596,40024077,40064522,40040241,46276326,1836462,19084880,40069977,997899,40009468,1790730,40161021,19017433,19027252,40224847,46287355,19047894,19128638,19122220,19006387,46287308,19118313,40227698,19055001,40151837,40229266,1796438,35603623,19030255,920301,19070435,1777288,42901627,1797521,40069978,19105420,19113610,46287421,19007703,40100125,1728608,19133882,19101403,19116438,19081058,19093973,40087818,19104979,19054961,35605341,40009669,19085982,19039090,1786621,40054330,40078282,42901509,19133891,19129788,19074699,19133913,19060198,46275622,19075034,40173650,40067084,40229696,19005835,40025077,1705709,19118476,35604163,40081790,19018117,920293,40182268,46287305,40232577,35605614,40067088,46275445,19010425,
 19085023,40009506,19133888,1747396,19112884,40131075,40071655,40147963,19121072,1750723,40231987,1712551,40064154,
 19112006,40021629,1729848,19133859,1729789,19123587,35604117,1738669,19047378,40231120,40009486,40231990,19107155,
 19106634,19063879,19072281,1775802,19025868,40232619,42707326,19024443,19002077,19104626,19019481,40009504,19060617,
 19006382,19038923,19111566,19046605,19111308,1724699,19006688,19002938,19074905,19001905,19116119,40029819,40239200,19100716,19101091,19006465,40167253,19120438,1748975,19038387,19063105,42899418,19087052,40081391,40238949,1742258,40016503,40168172,19098931,19044811,19031446,1743227,1760056,1734104,1728576,19085024,40024354,40024386,40084922,40116144,46275430,40009672,19112166,1786661,19075421,19112889,1775741,35603138,19031744,19083575,35604892,40224845,1724671,19129036,40065092,40040890,40108857,40081357,19112417,40074241,35605612,42799279,40069979,35605992,19125435,19126543,1713417,1713982,1708157,40024028,1736890,40241903,1740552,19113526,45775717,19031302,40024040,19029144,19027251,19111312,1747006,40157523,19111918,19001220,19134709,40220484,1773409,46287316,19074841,40028723,19083554,19125926,1769601,19105350,46287414,19118368,40163187,40170273,40232629,19123585,40039204,40085223,40021248,1594521,19106410,40058686,19106637,40009492,19115099,1713984,40236528,40029798,19006650,19129061,46287413,35604792,1750729,19107378,19006349,19137362,19001222,40008349,40024079,1790728,19072885,19085943,19083543,19116055,19006424,40234557,19023564,40009222,19115409,43011430,19039121,19120784,19109209,40092580,1749091,19045281,19061487,19115416,19071688,19128020,19111315,1790698,19128709,40164954,19047002,19127567,19001285,1768902,40229701,19112557,19078815,46275527,997932,19117799,19086078,19063880,1742432,40236837,40147069,40063680,1729720,19075422,19006385,46275425,19038492,1750731,19112126,40080972,19006461,19025826,19063932,19115159,19025851,19126338,19006415,1776783,40064545,40024035,19047259,42707533,19088924,35604068,1777283,19072881,19079749,40102223,19034075,46275102,19115428,42707536,19030032,40230702,19120203,40220483,1702364,19032895,40168046,40013082,40105357,19112044,19101373,40163104,40182272,19112885,1778257,1797516,19133900,40157521,40042741,19107379,40021311,19031741,19029143,19017229,19092357,19015181,904356,1713668,40073115,40009463,19103700,40164951,19047584,40021626,19030411,40063083,35606372,40024076,1724733,40236526,42800262,19134491,19105351,19128549,1790725,19115197,1836251,1836452,40092166,40040324,1702911,19105157,19115427,19074877,19060325,19000820,40064156,19017435,40041244,19082484,45775279,19128682,19116064,19018742,19075035,19038429,40092162,19117653,46275100,19107552,40040321,1796455,904512,19133901,997512,19036515,19121155,40220482,40169012,997981,1729844,19100438,19043984,19024416,45892607,19018878,44506691,1713733,19105306,19083577,19068758,1786877,19030163,19087271,40066773,40159303,40173353,40081405,40164950,46287327,19063993,44818081,19091610,40087810,1790868,46275706,1797557,40111166,46234353,19024448,19075039,19047074,19038430,19133328,43011439,19072256,1724711,1701676,19046427,19075425,40170272,40169550,19074872,40067091,19133886,35605335,19092721,40058964,35605324,40121176,1705711,19036474,19116086,40009664,1768851,40009494,19120125,46276318,19116097,40173631,1717505,35604109,40030606,40016504,19117798,40145355,1728603,19116063,19031594,40091408,19001349,40238195,35606423,40232627,922669,19024415,19129368,19113415,19136162,19006595,19105301,19063874,40157522,19001219,1786744,19038428,40087819,19087039,40157524,19001325,42902943,1747297,40123982,19116642,19105870,19009351,46287367,19040628,19030536,19136430,19027092,19107965,46275101,46287334,19115230,19092456,1724737,42901523,46275514,19112302,1713641,19001342,35606441,19087026,40059362,19021574,40125432,19111049,40229267,19011225,40182270,19105361,19056554,35604005,19006343,46287381,19133890,19001344,45892716,19009164,40229269,1713694,19117749,19107406,40147380,40169396,1724732,35604159,35605239,1713478,1836471,1708156,40238691,1747007,46287344,19133871,40021361,40009471,46287323,40081016,40173658,40232394,19061490,19038548,46287357,19017439,19113550,19113237,19060199,19074409,19082289,40170393,1702934,19063903,40092828,19057790,19100291,19021637,19025823,40229262,19116894,19006834,19128423,19133021,920300,19074935,1747040,40173948,40009489,19047893,922668,19030416,40163181,19107655,19019852,19104321,19133874,40009473,19088758,40170258,19129364,1769599,19021886,19037785,19006599,35603621,1705710,19030162,1713639,40067089,1786850,1734107,1729849,40081388,46275615,44814370,1728544,19037563,19064998,19117787,46287347,40081384,19061255,19106619,1716908,40016505,19133879,46234349,40073116,42900904,1747032,40021308,1768850,1768873,19106676,19032307,19084435,40009469,1836466,19136044,19123604,19105317,19081028,19006426,19105355,40089648,40182618,19123240,904450,40229276,1728860,19027227,40009501,19021278,19006388,35606373,19107154,19048603,40023777,19117397,19107453,19100345,920341,40121117,19071687,1740553,40236518,19084378,19123599,19018877,40169668,40229263,19055002,19116892,19078648,40231992,40170253,19111024,19102518,1738368,40024096,1724693,19048947,43525588,19118702,1776758,40053217,19116080,1746143,40100989,19098934,1786875,19048478,19058684,19113148,19117803,19085090,1708159,19103417,1747034,19110816,19050771,19047008,40105359,1796477,40232609,19080954,19059029,1836449,1774538,19105353,19101078,40228992,1790722,19117401,19020466,40009220,40058688,19017327,1778162,19001323,40161196,1790701,1717434,1724667,19041153,40009484,19109220,19104339,19050773,1729743,40025078,19087370,1742255,19117404,19075036,35606422,19060129,46287324,40170392,46287337,40182614,19092700,1769538,19030037,1796456,40092593,19078893,19102024,19107099,19035317,19117655,19100234,42900907,1836460,19105296,1836996,40028350,19133904,40232620,1705676,40025076,19133325,1593759,40161176,1733766,40236527,19116061,19047073,19099316,19007677,19106636,19105354,42902779,40032682,1594524,19134696,19109928,40108152,19136425,1786658,19073777,45774464,19116054,19058122,42899815,19104322,40173947,42900908,40172570,19103518,1790727,19036547,19106639,19038390,46287311,19064250,40009485,19108235,40096061,40064520,19032899,19136431,40041242,40132030,40025075,19048061,1713905,46233982,46234081,19084034,40021299,40163103,19107143,19104535,19063138,1747330,19115160,35603321,40067076,19126622,1741122,40009466,19087401,1768910,40182273,40009498,40092541,1734211,1738579,40230602,19061822,19101079,40092562,19086139,19117131,40170252,40236524,19106167,40042742,1729790,1786687,1728942,40009214,19117925,19128405,46287320,19006728,40173654,19134005,40029799,19023303,1746151,44814369,1768906,1734132,1741125,40182264,19009163,1836408,19113524,19111567,43525735,1713956,40025074,40081024,19031915,19111028,1738398,40020625,1796452,19133866,19109405,1836989,40132031,19081762,19116075,40024038,1768932,40232602,1705707,40241525,40234553,19085022,19084718,46287309,40092560,19084843,19116068,19128689,42708497,40081407,40008458,40230270,19109223,40170274,19010294,19107619,1716721,1715117,19107697,19114003,1713735,19116116,19060620,40091947,35604327,1713415,40102982,1786843,19091510,19038360,42901612,40009215,19111306,19073189,19027096,1708154,19112047,1836457,19047240,46275513,46275860,19094304,19006348,19046604,40164955,19118671,1738394,35606371,1736898,19112314,19017410,19060154,40020955,1733768,40069623,40236822,19072255,19116604,926066,40009499,1836246,40230265,19025827,45892576,40229698,46275707,1713700,19106096,46274210,40238958,40229259,19038388,1594525,40009474,19123886,1749133,40169395,40228404,19105339,1749061,1713547,19044424,40069625,19025869,42800400,46287418,19048474,40055902,19057561,40229279,40147349,19078422,19086824,19095177,40170261,40087948,19128774,35606439,1705740,40229695,19118560,19017431,19116089,40065480,46287312,46275517,40158047,19001148,19132596,997973,46287433,35604628,45775453,904448,19117656,40087945,19128018,42900888,40009225,35604796,40021627,40066768,19029685,35604627,19128418,1708100,46287444,19105325,19100111,19121894,1713370,19071695,19001326,40023494,19113225,19074971,19018516,19080833,1713332,19113804,40168170,1705738,40059633,19113607,19027067,19035318,1713908,40021313,19027063,19105332,40087946,19116121,1705677,1724666,40123627,19111897,19006422,40101829,19087036,19103458,19088929,19080151,19120084,46275426,19129804,40131077,19006043,19021247,19104454,40170251,1708153,19117135,35604119,40013068,40041258,1716905,40067087,19020020,19115417,40241526,40232638,45774463,19117392,1836430,46275431,19006726,40082165,19030010,40087943,19120672,19092356,19050778,40155188,40155846,1036898,19129961,19109202,1729750,40081397,40081387,40238687,40170268,1716742,35604544,19111533,19112562,1713591,19047006,40182278,19111323,990071,904604,40066775,40021306,40095869,19128751,19088430,19123648,40230701,19086825,19050777,40027293,19116893,40180925,19128131,19006598,19047075,46287386,19134008,1705703,1836431,19101963,35606440,46287353,40025468,19083687,40000287,1709034,40229265,1790723,40067085,1749057,19101616,19036514,19113605,19134007,35603622,45774861,19117402,19098891,19001255,19113079,19063162,35606488,1747326,19074700,40027289,40024683,19104508,19113614,19027093,44818079,40178234,19015123,19120875,1800856,46275432,1786873,35606013,19113078,1750500,19133907,40077120,40231988,19117662,40081356,19133894,904605,46234522,40028726,1836948,40081373,19092353,19001147,40170805,19088489,40227734,19040096,40232637,1763258,1790729,40232579,40170787,40239157,19105348,40232614,40087947,19086783,40131076,19103729,19072122,1786844,40229260,40024426,40236821,1236499,40024078,1713587,46287336,40131078,19123644,19074846,35605338,40171999,19035315,19082973,40016518,40241904,19109929,19080706,46275859,19136483,40234556,19106618,19117794,40099638,40161034,1713477,40027287,19074844,19017438,19116123,19027091,40146979,19116081,40232618,19131148,19001221,40021360,40131073,40001081,40237070,1771217,1705733,45774864,45776570,1728642,19060876,19022975,42874171,1713474,40029796,19011482,19096351,19017828,19075033,40173649,19103494,40008471,19116077,1708904,45774461,19081739,19071690,19006413,40238692,40008348,19103316,19046790,19081029,19115442,19093977,19024446,1776684,19017432,19117800,19117661,19133876,45775449,19024413,19129291,46275705,19095073,40040317,19126340,40081379,46287365,35606008,40106351,19016924,40021310,40232597,40009482,19058121,35605334,40024030,19111921,40092827,1713766,19093363,19111919,19000821,46287363,44818401,19100113,42800401,40229700,19103459,19123640,45774466,19087051,19057996,1836450,19006430,46287359,40224846,1836392,1705705,19074937,40059363,40238951,19064499,19009139,40231991,997501,40065479,40071653,40241901,1701651,19027065,1713416,19074848,19116070,19109406,40118618,19074936,19074932,1836403,19085867,45774459,46287306,40232639,19030033,40067075,19027066,19113077,1768904,19098879,40228993,46234344,1713444,1797560,19010400,40072608,19016166,1790699,46287328,19098180,35604545,19010202,19091153,1786654,1800855,19102105,40021624,19128748,35606002,40092594,19107621,19112886,19047892,1796458,19117926,1713739,19083544,40173657,19107974,1708152,19033343,19101672,1713697,40000290,40232543,40227699,19086760,46287330,1786847,40066896,1797558,40021307,1836241,40009475,40081782,19123605,40062184,40040242,1743224,19133878,19123600,19029377,19061032,43011440,40232632,19082453,19091609,1796475,19131484,19109201,40049202,1713445,42902191,1836451,40173626,40232631,19084282,1747041,40024442,40024034,1728605,42901629,19081056,19125201,1786688,46275104,40182611,1729980,1786660,920334,40170264,19031817,19131145,19112883,19016164,40107740,19006516,1747296,19028286,1709032,40229273,40168169,42706093,1713521,19096737,19111050,19030009,40009483,40024723,40081378,19023254,1740551,1836978,1797556,40025469,19074819,19021108,40163183,42708421,40071657,19063556,19054966,19104358,1730002,1747011,19120206,1836455,19109930,1747039,40234551,19087655,40230268,19082320,46276319,19099311,40069975,19048063,40081002,44506536,19116062,1708101,1713640,19116891,40064151,19111974,1836995,19112471,43526250,42800398,19062182,1837003,35605323,35606424,19087025,1713420,40170257,19087023,1786876,19019417,19137366,19099177,19113549,19116069,19096058,19030133,40064153,43526251,19108786,46275435,19105326,904449,19016927,1796439,40229280,46275623,19007676,1836432,19021109,19017440,19105309,1738366,1759877,19132315,19006045,40021303,40021593,19002936,19091631,40092165,19074934,46287443,19128705,35604157,35606489,40242468,19087055,40108153,19129037,19117790,35604793,40149428,42899816,40182275,19085328,19025867,1777261,40009806,19073182,19116057,19116073,40229285,35604155,40232622,42902850,19030194,19090573,19108031,19104374,19089686,19006685,1705736,1769595,1713368,46234345,1750722,46275617,19100112,1786655,19118036,19130121,45775451,19111307,35606019,19112878,1729977,19087662,40182617,40081383,35606009,1778228,1746940,40040320,40161177,40160705,19128420,40009480,19123642,40027290,19133903,922667,19113450,40024722,40101870,19083494,40091058,46287382,1746118,40240572,44784832,1771221,46275524,19047265,19001149,1717327,923081,1836406,40241905,45775281,45774465,40064150,19115429,1747010,1763259,19113546,19092354,19074880,19033342,19133905,40238959,19131146,904557,19102335,19040629,40054331,42900889,19113231,19085270,45776572,19113226,19133902,19083986,1836467,1769535,19118660,45892577,40146978,19006463,42900909,19060194,40053216,1836454,1768933,40170269,19106620,19104337,1747397,19112877,19111318,19001216,1768909,19060221,35604799,19103462,1708880,40173662,19035120,40063674,40238950,40009217,45776154,1777287,40009195,19107698,40073114,19133950,19082456,40081400,19116059,40173629,40066771,40173627,19043885,19024414,40020624,19116058,43011782,19031743,40221183,19001228,40232624,40121175,40173351,40009472,40077118,40024095,19063106,1836391,40021592,19041106,19046743,19082994,46275424,19025822,1738397,19133949,19031660,42901525,19133867,40009213,40173652,46233939,46234524,19105305,19066439,19104295,19113234,19120710,19116122,40228859,19117394,19125927,19117133,40107738,40229275,40078281,40008460,19113466,40027291,19085136,46287361,40063021,19109218,40230269,19081055,19127611,40173628,19103530,40185086,40081374,19115066,1705732,40021314,19006420,19113611,19127541,1836398,1713475,40024424,40009219,19133870,1796453,19136481,1769598,40009490,19133860,19047071,19072260,19133018,19087250,46276327,19112007,46276325,19105801,19030260,40101830,40180928,19133865,45776343,40069621,19054969,40091056,19079186,19024420,19110838,19072520,35604890,19010426,1778166,19113236,19117792,35603858,40100266,46287354,45775714,44506535,40232621,1729847,40173651,19112008,1713476,19045211,1759851,19086784,1760054,19086823,19115171,35603323,1713699,19112870,40024036,45892954,1747298,19034239,19120269,19002935,40163179,1768941,40057581,19107411,35605996,19108239,19109451,35604004,40236523,19017464,19112057,19045284,40009505,40224207,40029800,40232607,1705704,40173666,46275619,19104453,40058689,40092578,1790697,19119679,19111025,19133861,1742288,19018013,1747295,19133022,1836459,19111313,19114555,19022244,904517,19114554,19116439,40069622,19102540,46234411,40069478,40146216,46275429,35606007,35606491,19128019,40021315,19133875,19115444,40229693,40173352,35604889,1768962,1836399,46275106,19136482,42901508,40231953,1786747,19117137,46275618,1713959,40238961,40090881,19088223,46275709,19117037,1768849,40081393,40063085,40170394,40238953,19116605,40230601,19010427,35603857,35606487,1705706,19031746,40229268,19111922,40105044,35606003,19065113,19117391,19047004,19133862,1702936,40052006,40163191,42899419,40087949,19024447,40013066,46287412,1778227,40132029,19107699,19101404,19133872,1713543,40168823,40040323,42706988,40238956,1729979,45775448,19095532,19074876,19115722,40167947,40027636,19070704,19100315,19096779,19063555,19073187,1747205,19072283,19115231,19110901,19034954,19105322,19113595,19133884,1836458,40182267,997881,19019415,35606010,40001160,19026199,40127529,19006727,19110900,35606419,40064521,19104318,19103286,19071423,19111563,19110817,40021304,19074975,19108075,35605322,19006429,40099639,19062304,40236077,40168168,19113371,40090883,904514,19106640,40085267,42903358,40231989,19133877,19082487,19105618,19075010,19115704,1778226,19105595,19133004,19073219,40052000,19006682,19109919,35605337,19117390,46234346,19123877,40009488,19074979,19103301,40040611,1771162,19062261,19120375,40236521,19117393,19112887,42874170,19045287,40180926,19030413,19128602,1713479,40024746,40029801,19129411,1734140,46234351,19048407,1786682,40155404,19080618,46275510,1800837,46287362,40232615,19109208,40127090,19111317,1836434,40169398,1713442,19117395,19115426,40024409,1790731,46287368,19010201,1769600,19045286,46275519,19064329,35606004,1750569,1768903,19111896,40238960,19102568,40173952,1759848,19088923,19104510,35604891,19097754,19031749,40170250,19076752,19105292,19024442,46275515,40084921,40067092,43525739,40092514,40009507,40024039,19079960,40102466,19133873,1796441,19087889,19105554,40011363,45776337,40101431,19111564,19111319,19094280,19086136,19027064,1773402,904520,19133893,46287310,40069973,19116067,19111477,19056714,40163194,46287379,40096044,19124564,19006566,40171393,19045256,1769540,40169397,1736892,19133864,19047197,42900887,40238688,46287364,19117824,40090882,46287352,1836433,40123620,19070174,19122188,46287366,1750724,40040325,46275509,1713419,1769562,1593760,19106222,42874173,19084536,40006341,937712,46287441,40146078,19105820,40060425,19118551,40169547,35606486,19108777,19103587,1708903,1750501,35604058,40220977,40102428,19063163,43011789,19102606,19074909,40229691,40232604,19074878,1747038,19133909,19046137,19112871,40180929,19099966,40020626,40155192,40173647,1716744,40136069,19079748,40229282,40029797,19032304,19133910,40129858,19006513,35604795,19063582,19113827,19110532,19118701,40168310,35603320,19061882,40063084,19113417,19085866,40009493,35604328,40169548,19111565,46287503,46275427,46275710,19105310,1743222,19030164,40007235,40024770,19098179,35605655,19108743,19009138,35606367,19027069,40232633,19079928,1705739,19128750,19006563,19111924,40092563,40069976,40024072,19118239,40108712,40232606,40029802,19045259,46275711,40027288,40013064,19104294,35605238,1709170,19030259,42901624,1786748,19107351,19044789,19105596,40081027,40064519,40232544,40236522,40081382,40178233,40166675,40229270,19105295,19009330,40101429,19118668,19030063,40024032,1709001,19111324,1836448,998012,19105597,19072282,40173630,19023679,19117789,19118037,40165695,1724741,46287314,1836447,19034079,19003029,40169544,19121051,19115425,19135068,40169228,35603324,19070701,1750727,19128706,40220673,19110534,19019858,1708961,1724697,19070873,19025824,19109259,1740547,19095678,19098874,19006722,19110902,19105643,1713734,19113544,19133895,35605822,40083994,40224844,19111316,19027068,40230606,19116066,1713767,19130134,19001290,46275877,40106350,35606421,1701674,1717408,35605998,1712573,19120870,19044417,1786749,19007702,1713337,19095060,1724735,19086071,40232603,40059632,19037562,19006414,40069666,40161022,1713413,40140006,19113543,19010505,40163895,1733765,19010702,40229257,45775686,19001150,19060200,1728606,19086014,40080973,19015125,1713414,35606368,46275876,40069972,19103288,19115140,40088433,19107993,19030510,19116079,19113615,19016349,46275523,40080968,40021628,40163186,40230266,19084380,42799280,19050775,19087656,40234615,46287384,19006350,40024682,46221511,40229283,19120087,19045285,40163177,990069,46287348,19123650,40185085,19111920,40126077,40001082,19047198,40167251,1778167,997979,19017436,19006044,19133015,19087054,19133326,19073779,19024418,19006416,40131074,40170254,40167250,35604790,1786652,19074901,40047623,19045253,19129063,40009479,1800836,19058103,19101718,40145358,19063875,19107464,19070253,19001350,1705741,42903381,1773432,1749082,19054963,19094616,19095660,1713338,19088224,40167948,1593763,19073811,40100126,19027095,19060193,40169227,19136423,45892657,35604797,19001213,44818078,19112059,40232628,1769541,19063140,40071743,46275624,19132594,19102890,19076407,19116087,19006423,19115226,19001348,19054936,19084281,19073186,19105568,19120202,19118669,40220671,904602,40232612,40228860,1796460,19113232,40021316,40081813,19070178,40180779,1836395,19073217,19030257,19109219,35606490,40221184,1742253,1738521,42899814,1712549,19110718,19074818,40182619,46234523,40170260,19083688,19024444,42706229,19111048,40067083,19115423,19110814,1836988,19038870,35605611,19063876,40039206,40231993,19027986,19041154,956658,904561,40238690,40173697,19034870,1713671,19112301,40053973,19117108,40173664,19087249,19115097,19088429,997934,19001341,46275621,19111309,42800399,19063877,1786745,19031816,40243127,19136021,40029803,19131817,19072858,1749090,19111053,19109955,19133898,40107545,46275521,40168821,19107454,40081399,40081787,40167956,19096019,19068363,19036472,46287417,904518,35604158,19116120,19115424,19027098,40000288,1747005,40091057,19116603,19058210,19091790,40236823,46234350,19100346,19113782,40168047,35605995,19116056,40009495,40081392,19002937,19092454,40081017,46275522,19080894,40009192,40024074,19074973,19113525,1729978,19074974,40232610,40065427,19031742,19016795,19030062,19031941,40167955,1713762,19085868,19081054,19096778,19017434,1786719,45892953,40173653,46275446,40173663,19109020,40009491,1749083,19060153,19018114,19115227,40168173,46275110,19060528,40229697,40125512,40024406,40088435,19081505,35603319,46221515,19128404,1836402,45776150,19129059,46234352,19120855,19006723,45774868,40173951,19032384,19006517,35603576,19103180,19096054,40170259,19006468,1729744,46287419,19133869,19074847,40170298,19025853,40000291,19025852,19073812,19125922,40032939,19123651,19016350,1729845,40081007,19034078,40067079,19035320,40098939,40057580,46275511,45892955,19133019,35603608,1717963,19058072,997980,19104340,1773411,19064500,1713544,42799247,19128422,19000817,19006389,19106232,19023670,19128707,40069624,19050776,40009476,19024417,19083152,19124327,19075379,44506690,19061513,1746150,19117400,19035316,19107466,19105345,1786874,19113735,40080971,40170270,19085170,46275616,1747241,40041279,19009162,19006681,1836446,19103152,40092513,1713957,19040627,1836973,19087402,19116065,40078283,19120838,19040630,40032664,40163189,19034955,1729846,40145357,40036245,45775447,19105338,19113532,40170265,35603137,40229694,40182274,40021625,19113606,40009477,19075032,19107479,42900886,35604162,19047377,19095074,40000285,40081358,1713738,1724703,1742254,19017437,19122179,1750730,19118124,1713732,40016502,19016165,40009193,35603609,35603325,1750725,19056713,19037561,1768940,35605999,40040316,19116938,1790024,19048121,19133887,19001254,19110531,46275518,40182612,1836407,19128124,19110533,46275428,40092163,19075038,19117801,45775280,1836400,19133897,40221185,40105048,19070175,19006421,42706230,1768881,19011226,40163190,19123639,1797513,40074251,19134595,19128421,40009508,19050772,19099300,40169667,997496,40092595,19006567,1747454,40009194,35606000,40081789,19116076,40147962,19031748,19133885,42800263,40161186,1721543,40021250,19123591,40067095,19030258,40009487,19133953,40008345,40009500,45776571,19063878,1146884,1750568,1734133,19128842,40067093,42874174,19116088,40071758,1786684,19089978,1836401,40078280,19117165,19111027,1734207,1594523,19117035,19109021,19129192,1797515,19105761,40021312,19075606,19073185,19117802,19101080,1786842,43525583,19070179,19044547,19103138,19115088,19074972,42899812,19072883,19024449,19082972,1836469,19010428,19075380,1734232,1713737,40079525,1790692,40064524,40024342,40081783,19060405,19113781,40170395,35605994,19112303,19000818,920335,925952,19085757,19018156,1705752,40163192,19129058,40229692,19100631,40056693,19115414,19031818,40009665,40229281,19073184,35604070,1750728,35604156,19106168,19016925,19120380,19109404,40021305,40054719,35606005,1747209,19006046,40080750,42900910,19030039,19003644,19128975,40100988,19010709,43526249,19109216,40071410,1774932,19088225,19076624,19006515,1729783,46287335,40232635,19034240,1746119,40173354,1836409,19017326,19113818,40013081,19136020,40105358,19123606,19105759,19047195,19127631,40079524,40105046,19031596,40234554,40066772,19104619,19006464,19100235,19120085,1790726,40220676,1713736,1729745,19088795,19086790,19001229,920299,19001288,19121615,19068726,1797532,1747294,19133013,1738393,19006345,19001215,19113533,1734205,40182613,19118314,42709124,19105869,19028241,19123603,40009481,19023669,19116103,19029718,40180930,45775736,40173949,1713961,40052007,40147374,19073181,46287338,40008561,40069653,45775719,19117660,1777285,40021317,19006347,45775713,40221182,19115443,1734210,46287315,19128639,19112565,19002939,19133858,19047379,35606442,19011873,46275875,19042751,19019411,40021251,19098935,40230605,19111157,19003645,19117923,35605993,19115703,19027226,1705735,19105319,1749056,19072857,40145356,19074933,19102859,19074843,19095677,40024353,1746121,40232542,19075001,19016348,1736887,19109931,46287340,19075031,1837001,19006600,19117657,1800835,19122045,35604791,19112046,46287377,19117398,19083578,1729873,19079805,19120082,19116072,40224850,19106524,19052683,19123962,19133922,40161035,19123963,1768935,1713693,19135069,19078892,40008350,19001214,19034882,19136019,19101717,35604154)

In [ ]:
ccb=(19128152,1319139,40171563,19036447,19120546,19120547,19101994,19134542,40171548,19116934,1319943,19112983,19134904,40167847,19134240,1319975,19116740,19127430,19019310,42629420,19128148,45892992,19104027,1318859,19016046,19117025,19097791,19054614,1319906,40063053,1318918,19128153,40069599,19134239,19102106,40068432,19081867,19116991,19036450,1332500,1332522,19016611,1319133,40041997,40168081,40171554,40165780,40171549,19134278,19025502,1319880,40069238,40068430,19036471,19054617,19016347,19112979,1319163,1345166,19113736,19103795,40226396,19107499,19044936,19071553,40160343,19110673,19017500,1332450,40037281,42629415,1318983,19134730,19076413,43526898,1345165,40226376,19134815,19054524,1326082,42629421,1318137,19103844,42629411,19129588,19115309,40171557,19128149,19025504,40037298,19116587,40068434,40167838,19134404,40171550,19015804,1332528,19116817,40068781,1332421,40165074,43526895,1319972,19134276,19134405,19019331,40008977,40165082,1312776,40068771,19071510,1332529,19134261,40167801,19134816,1318860,40226395,40165077,40224174,19004572,19099836,40167853,1319979,19134539,19003921,19106594,40224173,1719007,19016488,40056270,40068433,19015802,19019333,19062267,19105732,40165748,19110976,40165087,40008942,974883,19117300,40224167,42629419,40069552,19134257,19017087,40069145,19120542,19120502,19134561,19134645,40165093,40069151,40224154,19020063,40232340,19116980,19134749,19025440,40037280,19117146,1319162,40069551,1319166,40167852,19120548,40224169,40224178,1318953,19025506,40224175,1319946,1318956,19134403,42629416,1332497,40165099,19020110,19134657,1319947,19020067,1312778,19107546,42629417,40165073,40066864,40068775,40224171,1312777,40160338,45892991,40145312,40165069,19128164,19125704,1353780,19107123,40010569,19044935,19054217,1719009,40068435,19019309,19104654,19134280,19128168,1318914,19020065,19134264,19127561,1319950,40232338,19112982,19134254,19016590,40232343,19134275,40167844,40010568,40165088,19129157,19134545,40232332,19118469,19107317,19033959,19117604,40008978,19046451,1319136,40165094,40069154,19113737,40052888,19020166,19112984,19134905,19054523,1345164,40068783,19019334,19080313,1332531,1353820,19103475,1319944,19108892,40165078,1319973,19054091,1319949,19134250,19134236,19128162,19117065,40165064,40232329,19101748,19134813,40066869,40065688,19011549,19103936,40163978,19106593,40068774,19128151,19102399,1319140,19014793,19134751,19048140,19134263,40232339,40165075,40226372,19107812,1318985,19129584,40069146,40068439,19012420,1319141,19128150,40224172,19054554,40069155,40140541,19134248,19004539,19117161,1319976,19010264,19117126,19128158,19128167,19134817,40061782,40163965,40168082,19134244,19020061,19117299,45892985,19128154,40165070,19116572,40165090,19117147,40224166,40069237,19017657,45892988,40168737,19071995,19061715,1319903,1318916,1319942,1319977,19020066,1319945,19036476,40171547,1318987,19128166,19134279,19003085,19134249,19134237,19101798,19112981,40167845,19129757,19127432,19116595,40069156,19129156,1332526,40102956,19048223,40163963,40058618,40118536,19024112,40141364,19054522,40167854,19018115,40226391,40171562,19080314,1332420,40054848,40226392,19102567,19115201,19064962,42628858,19134241,40068442,40068443,974884,40068444,40224168,19000879,40068446,40142680,19117301,19017409,19127434,40068436,40118367,40232336,19025505,42628857,40171559,19046429,40083419,19134811,40232334,19134282,42629412,40167850,1318913,19024115,1318912,19110972,19134566,40232333,40226377,19117302,40232331,40069147,1719008,19129155,19134558,40171561,40168083,40165081,19048136,40224177,19135441,19104018,19038993,40165778,19134564,40165076,40069153,40010567,1319951,974886,19076412,19134812,19108217,40052890,19116960,40058588,1332448,1345162,1319974,40165781,19134546,19061252,40232337,40163964,40008976,42629414,19128165,40069600,42628860,42629422,1319908,19128159,40171552,1318957,19036443,40167839,1319164,19134251,1326021,19134902,19054616,19110672,40165092,40063052,40163977,19014769,1319135,40171558,1345141,40068428,19078726,1353818,40171553,40003426,19125385,1326014,40068427,19117603,19117212,1353815,1319909,19054525,19103937,19134750,40165068,42628866,40171556,974860,40171555,974859,40069149,19128163,19115858,1345163,40039294,19061826,19134255,1318988,1332442,19073093,1332527,40029293,40069553,42629413,19020157,40167848,19025501,1318952,40102957,19122209,974858,19102396,45892984,1319978,19134540,42628859,19117607,1318164,40068429,19134541,40165079,40232344,40224179,40232330,40058099,19120491,19014767,19116982,40069235,19020162,19103794,19033958,19134238,40232328,19080315,19102397,1319138,19110994,19127433,40069144,1318167,19012824,40167802,1332418,19048138,19117602,40068445,19093207,42628863,19117070,40224170,19108891,1318917,19134262,40167846,19134543,40165089,19134260,19134544,40171564,19061714,19104017,40165085,19019332,19010577,40066868,19078727,40165065,19054270,19116850,19011548,40168739,19013393,19134230,19003924,40068424,19134259,1319902,1312779,19024113,19134222,40068437,40226375,19015806,19003925,19128157,1318853,1319910,1353776,40165066,19081869,19120471,40069152,40165098,1326085,19099857,19016969,19081870,19089969,1332494,1326015,19073094,40069148,40226366,40168497,40069150,40171551,40066837,19016588,19128156,40167851,1332419,43526897,19118473,40167849,19128155,40167843,40144458,19117608,19111022,1319881,40066870,19122287,40068441,19115308,40168080,19122210,40165096,40123872,40168736,40171560,19003922,1345167,19117606,1319905,1332530,19117044,45892981,19020064,19107370,19128160,40008979,40226374,19129737,19115856,1318887,40168738,43526899,1318888,19016346,19134538,19116624,1319948,40165063,1332525,19102566,19017059,19020062,19017521,19128169,19134258,1318915,1332532,43526896,1332499,40142669,19120539,40165086,42628864,40142670,40069234,1353779,19112980,19044937,19134277,40224180,40165091,40226394,40055561,1326020,42628865,19113063,19003890,19048172,974887,40224176,19016653,1319907,40068776,40232341,1353816,40232335,19046430,19129614,974861,40052891,40168498,40165779,19062645,1318921,19107612,40058587,19081891,40165062,40102980,19098344,40068426,40165097,19134547,45892993,19071509,40037291,19061429,19116981,19115729,40068440,19036449,1318919,40165095,42629418,40068785,40068431,45892980,40052889,19025503,40068447,1318984,40165080,19101807,1326012,40069236,1319904,19134909,19081893,19116885,1326083,40226393,19128161,40163962,19134903,40008941,40165782,19115857,19014791,19024114,974885,19025439,19116935,19108731,19016869,19010379,19081868,40226373,1319165,40165084,1332449,19103832,19091506,19110992,1332495,40232342,45892983,40052887,19134270,45892987,19120540,40165783,40163961,1318989,19134908,19117605,40068438,40069157,19098343,40165083,19134281,19104028,19110980,19000880,1326084,1353817,19017656,40165067,1326013,19010432,1332496,45892989,40224153,19134548,19134256,40226368,19115260)

In [ ]:
diuretics=(40165771,35605440,40226401,992591,974423,19010015,19062385,948821,992646,40163758,40011843,974431,19053108,40084699,974430,19075153,40174795,19078353,19053106,40174799,40045754,974861,905275,40044846,19132619,19039348,19039749,19082886,19034479,40044730,40226382,40059140,19019232,40174807,40045765,40044449,40020438,19029343,40224176,40007368,40184213,40044459,40232335,40232341,40010558,19057855,19057857,42800843,974200,40165759,19107536,40178452,19059722,19078351,19039350,40174783,40012106,42800845,1316400,19106368,40044461,40026133,19075944,40165760,1316422,40065753,992674,19069124,40232342,40107746,40178460,19081455,40171658,19128391,40028116,19082437,974887,40044836,40177828,40045760,19048172,19036920,19057853,19037386,40044853,40045119,19034475,40044453,19132831,19112752,19065802,40239562,40174784,40177818,40184217,40226381,40224153,19132613,40163111,974171,40044458,40039209,19023454,40166822,19069037,19063078,19036919,40011859,40044817,974166,40171670,19037084,40162876,992590,19031850,19106626,19121017,40174790,40184214,40133921,43013685,974885,40163750,19036922,974474,904662,19115265,19023733,40073718,19036917,974447,40115014,40115467,19071685,40184247,904548,904640,974711,974223,40027430,40059139,40171679,40012101,19037521,948790,974429,40169649,40171668,19036929,974852,19023737,992673,40163121,40171680,974853,40174779,40174778,40147135,40163120,40065754,19073983,42707385,40167096,40166821,19135232,19128395,40045756,974648,19118542,974227,40147525,19062414,40167095,40224167,19106340,974883,40166820,40174806,40184184,19003733,40165757,40159174,19107969,904598,19005704,19117865,40020718,19037249,40162412,19132615,19073982,40045753,19117230,40020715,40169643,974620,19039389,40174804,19106767,40050442,1316356,19081452,19124246,974427,40223168,19073061,40050455,40142839,19106624,40178461,19037488,40096153,19132612,19019156,40012100,40044834,40026132,948817,991420,19109563,992648,40050440,905277,40162769,19045562,19112606,974826,40163113,40162865,19089074,35605439,40167094,19106550,40167092,19063004,40044856,40163109,991390,40045124,40163122,19117225,40020555,42800847,42800852,40045118,40165787,40184190,974226,40007379,19106336,40162416,40162864,19106329,40044460,40160343,40163119,40184246,40226398,19087457,40012108,42800842,19117227,19081428,19112252,19062387,19118541,40044466,40045122,905276,19078161,19078101,974642,974170,40115015,905306,19037459,904702,1326443,1376289,40171669,40133310,40163126,19081678,19107173,40044451,19081453,40171660,19037528,19037187,19069032,40174780,40044824,40020719,40171681,40227654,992616,40174801,948791,19039382,19078106,40167113,19069128,40044816,40224174,19101256,42800844,40226379,992617,40162414,40227651,40167118,19078105,19063050,19081456,40167122,19125472,19096059,1316360,19132618,40224173,42800846,19112053,40171666,40184278,19115794,19125473,40162872,42707386,40232340,19098223,40011842,40224154,19065130,40167091,19131968,19005705,43013683,40165786,19113098,19131972,40065752,42707378,19027600,40033920,19087507,19117229,40065738,40143132,40162413,40224175,19077922,40224178,40044849,19133002,40169652,19106242,40102925,905308,992649,19036921,40224169,40044452,19135223,974224,19037452,40163106,19046549,905278,40184187,40226383,19106520,19129687,40243938,19106900,974442,40174802,40007381,19075942,19081451,40054950,19062609,40160338,40224171,40163748,40133318,40174797,40166817,19037455,19080537,992620,19081475,40020435,19063049,974443,40003424,19086977,40044831,40226380,19106456,19069100,19129686,19003603,948789,40169650,19030022,992644,19096771,19135748,40174793,40224180,19061980,948787,19131971,40232338,974828,19106539,19005703,974732,19078166,40007383,904549,19117231,40044838,40054954,19003606,40007372,19132620,40044822,40174800,40232343,19027629,40163757,40174820,19064259,974192,19135231,19107539,19132833,40011857,19064257,19124247,40222167,19107112,40232332,19078137,40042000,40184188,19102493,974740,40020717,19115793,40154690,40045126,19019157,19111605,19036918,19073985,40045762,19069033,19045770,19106534,40011850,19061885,1376291,19107614,19069038,40226402,19135233,19113825,40184154,19106269,40232329,40174782,40045128,948855,19023735,19034474,19021220,40073742,19003578,19045846,40065737,19037083,40168022,19119996,19039387,40044839,40174808,40020558,19036924,40162866,40073741,19047861,40177821,19117224,974199,19039381,19109702,19132611,19048140,974473,40232339,19029345,40168021,40165785,40163118,19023731,40226399,19062419,19118719,974678,40177822,19029062,40232337,40011851,19121016,19019044,19131518,19062389,19118150,40015092,40166819,40163115,40020439,40085945,19037458,904639,19045569,40045761,19078165,40044852,40044818,40167125,40224172,19032893,19096859,19115829,19005702,19022486,19078352,19023738,19082888,19036928,19114075,40044850,19032303,19107180,948819,40174789,19080037,40044450,40174821,19106768,40171665,40167115,19132834,40224166,40226378,19102492,40044456,40243939,40174796,40065739,19037489,19112050,19029346,40045757,40166818,19037457,19003516,992647,19062415,19128392,948820,40162875,19062413,19107596,40011854,19087459,40163759,19045561,974169,19032921,19062610,19010018,19003737,19129688,19048223,40044457,40184191,40007385,974741,40007369,19081454,19081853,42800850,19071755,40162415,40045116,974201,974884,19037456,40163124,40184185,19081509,40044825,40084700,19128393,40011860,974647,40012102,992614,904597,19061818,40174798,19039388,40054952,19047862,40007370,19023732,40073717,974193,40133317,40174781,40174819,40028110,40174805,19135222,19037085,40224168,40044857,19118720,19003828,40050439,974739,40178462,19132616,974710,974225,19019046,40232334,19035411,40045759,40044819,19042717,40174803,40050457,19037451,40178451,19081429,40044842,992621,40232331,40044848,40060233,40044841,40163125,40044729,40162873,974643,40011846,19109413,40044845,19131973,40174794,19061678,40044821,40044843,19118501,19131970,19048136,40162871,40154695,19023453,19135441,40224177,19078138,19058026,974194,40011856,19042653,19103824,974886,1316355,40044465,19106549,19092316,40184277,19072252,40174769,40073719,19106363,40177819,40155317,19079579,19003517,974428,40167090,40054948,40044832,40232333,19121339,40172078,40232336,40167087,974854,19062341,974677,19010842,992612,40044454,40226397,40044854,40011858,40050456,19029342,40020716,904671,40065740,19132999,974198,40163117,19073065,40226400,19131969,19021262,19029337,905273,19042651,19118531,948818,974860,43013684,40045115,40167119,19003126,40184186,19128390,974859,40167121,40085943,40044462,19106625,40045758,19106648,19039349,19114076,40085941,974196,40012099,19076046,40167093,19113895,40167124,19069122,904547,40174770,40095410,40162877,19059690,40154112,19063044,974702,40163123,40163752,19073063,974858,1316354,40169648,19021264,19078080,19107538,40155320,19078104,40232330,19037453,40174785,40224179,19115264,19108129,40232344,40174792,19132617,992618,19037384,40007382,40012107,40174791,40034892,40232328,19106538,19080539,19117226,40163116,19087506,40050441,19135230,40044455,19031848,19048138,40085944,19019231,40044837,19107669,19081474,40011845,40028109,19016929,40177826,974709,40224170,40073715,40165750,40011853,19042713,19032898,40171667,42800851,19054951,1316401,40174771,40176023,40011855,19019933,40174822,19079429,19064241,19036927,19023739,974827,40115468,19089075,19132614,40174809,19108929,19107339,40044851,19115736,40163114,19075873,40163112,40167088,40085942,948816,974645,19045568,40045764,40020436,19010016,40045755,19073986,19109416,19109018,19019668,40162417,19036923,40012103,19036926,40171659,974195,19132832,19070766,19102491,40174762,40137849,19107338,40184216,19045567,40155308,19078139,40184189,19113099,40167089,19032892,40007378,19080032,40177827,1316359,19037460,40147529,19081430,40184215,19037385,992613,40007377,40027598,19107491,40174786,19117228,19111679,902518,1376290,40011852,40045763,19036925,19023736,19069127,19003738,40045117,19078140,40060227,40162766,19045565,40045121,40169651,40007380,19029348,19080010,40011844,40133316,40163751,19035389,19023734,19113826)

In [ ]:
opioids=(40169868,19107320,40062123,1103423,42707626,40173137,19068248,40148022,42903345,40232746,40170019,19086196,40169923,40231882,35604843,40167711,40169871,40243245,40067525,42903219,40160459,40173114,40169339,40166395,40169925,19133292,40169346,40175253,40176085,40232709,19117870,44818050,40170027,46287471,40231921,40232740,1129106,19063889,19119637,40243122,40089949,40129005,19115064,40169916,42707616,35603624,35604831,40176086,1154029,40169934,40155160,42874042,46234298,40169993,40173117,40243411,40173072,42874031,40063128,19115712,42705321,19134115,40232716,1718699,19044083,40165012,40135194,40243100,42707721,40170068,1154280,40012599,42707730,40173064,40023610,1103352,1592363,1592364,40169195,1718690,42903006,42707737,19116025,40089948,40231887,19115392,46287470,19016733,42707754,40164993,1153703,19098203,40171441,42707746,40169952,40242502,19006054,40231906,46287723,40169950,40072458,19046076,40162164,19002647,40169991,40056059,40169920,40231928,1154314,42707763,35604830,19006056,42707601,42705327,42628940,40169896,40176084,44814263,1153711,19134048,19002431,40231919,35605654,40169932,19062492,40231916,35605838,19016735,35603805,40147600,40167710,19006142,40061574,40168284,40072159,40221212,19024385,19098911,19129648,40002781,40171473,40176071,40169989,40231870,40150029,40164991,40231855,19063888,40223252,40169966,19060939,40169936,42705497,35605651,40173120,42705456,40231809,40231930,42705479,40171440,35603935,40223141,40169957,40243101,35603946,40232759,40169338,40061538,42705476,40242501,40169323,40165020,46287325,1153664,19006024,40170072,42707769,40004068,40169967,1719035,19016734,40166398,42874037,19006060,40170056,40170075,40061355,19077248,19123904,42901172,19135553,19135563,42705493,1189596,42628928,1111318,45775131,1133227,40061356,40170081,40076969,19026295,44818054,40231932,35605645,40221142,40169881,19061988,1125199,42901815,1125233,19123905,40232717,40169979,40232765,42705319,40221207,40221206,40243096,19135574,40166402,19133110,1592532,40239281,19001061,19135530,40232743,42707620,1153702,40170014,40166396,42705458,42902732,40240781,40221229,42705491,40170436,19084972,19102015,35603637,42903327,40063477,42707722,42902835,42901175,40232712,42707723,40170038,35603632,40061544,42628925,42903228,40169320,42705310,1125200,1103399,40232711,40173092,19006099,40242510,40173111,40051239,1154312,40171481,40173075,40146010,40166404,40164990,19017906,40169951,19135526,42707791,40176082,1718957,40169906,19063887,42705314,46287473,40170025,19002648,44814262,19135562,46287326,40072160,42902406,40170008,40169354,40163259,19135513,35605844,40231857,40165001,40076967,42874038,40076971,42705517,1592352,19134122,40002749,40173008,40176079,19098897,42705316,40169813,40169990,1133223,40169345,40242507,40070544,19006053,40166392,40231886,35603602,40169969,19083035,40169938,40170073,40173150,19134046,40163871,40077882,1103358,40070566,35603607,42705320,40232713,40169893,42903403,40012259,40173074,42628933,40067528,40176067,1154149,40240779,40243097,40173083,40170062,1130586,19060339,40095270,35604833,19133294,40165013,40061560,45892529,19105487,40165727,40234445,35605847,19135578,40221219,42705501,40232720,40061565,35603593,19134266,40171442,40231910,40232570,40173079,40166399,44814282,19006058,42705305,40231804,40165010,40169891,40221231,35603629,19115118,46287480,1114122,40238706,35603635,19135576,46234115,19135550,40056821,40089955,19012856,40165024,40231934,40231884,40169955,35605849,40231927,19062894,19116005,19111468,40169930,40231862,40171398,19135537,40169924,40169317,40181188,19031906,19077272,44507563,19135533,46287469,40221224,40095271,19107319,40173143,42705507,19012851,19135523,42705506,19135559,40076972,40170052,42902609,40162168,40231902,35603939,42705465,40232699,42707955,40232749,40067526,40173119,40231967,35605846,40231879,19003151,40151317,42707774,40149460,42903075,40147578,40164998,40221217,40170015,40221215,40012603,19135472,19133109,40174868,19135573,40089952,40231885,40232730,45774942,42902654,19006052,46287332,1103347,42628932,40067527,40232742,42903287,40072161,40173154,40002751,46234113,40168285,19134125,40174866,40169900,46221484,40231819,40231864,42873587,19135623,19059093,40003545,40232753,44814283,1153705,42705505,40164999,42707606,1592527,40231944,19017164,40072158,35604822,35606052,35605658,40169879,1113754,40067524,40002750,35604935,40171477,40232714,19100090,35605836,40169328,40169306,40227615,40168602,35603937,40169986,40232707,40155170,40231925,40162163,40072157,19017909,19001091,40136809,19135577,40165018,42707618,40169892,40221198,42707622,45774948,19086191,40166393,44814281,40160378,40232705,40051245,40243115,40231922,1130614,19134265,19134109,42707729,19041082,40149758,40169964,42705512,40169908,40231881,1135463,40173151,40221211,35605841,40221226,40170035,40169334,42902958,40170055,19063337,19016516,46287286,35604837,40169944,40231913,40227601,40239283,40231933,19017557,1110410,19112164,42705308,40232734,40170040,19054211,40165716,19044950,40231918,19134269,40243121,40167712,40240780,42707751,19116404,40171439,40169935,40232758,40002755,19061281,40221205,40176032,35605845,1593598,42705325,42902408,40166403,40051242,40174864,1135447,40170001,42707608,19006028,40173097,40173055,40169915,1129030,19083552,40034969,1103400,40170074,42707773,40169888,40067523,35603630,40173125,46287479,40231914,42899096,40051248,1593599,42628935,1154151,19134267,40169322,1126658,1154278,19134110,35603943,40231929,40169953,40063475,19022056,42707738,40163873,1133732,19021940,46287472,40012596,40160379,19135567,35604847,1593600,35603947,1111321,42628931,42874032,40174860,40155880,35604841,40070564,19002650,1113731,19088666,40168604,40012258,40170061,40089958,40072456,42707623,40169321,46287477,40169914,40243116,40169904,19006051,40170042,40169931,1103424,44814193,40169919,40002780,35603938,40243112,19133844,40173472,40170077,42707756,40232751,40228550,42707621,42628926,40175248,19134118,40163258,40170060,40231904,1103354,40034820,40154651,40034819,40169973,40165014,40173115,40169974,19087129,40090252,35603580,35603944,1154315,40174861,42707726,35603599,35605839,40234431,40169333,1592668,40232739,40173155,40169330,42628941,40231912,42902815,40173468,35605646,40148438,40011279,40232725,40061392,19002986,40147575,40169876,42707605,40166406,40174852,19135622,1125035,40173100,1592358,40233247,40174863,40232763,42707614,40061572,40175252,42901174,40089954,40176034,42903376,19134126,40061568,40012255,19009299,40174862,40223205,40166400,19001092,1153732,40232715,40227591,19116003,40170011,19135554,19088417,1153671,40173091,40118178,42873361,40165722,19134120,40165017,1592354,40173136,40169902,40231856,1129189,40242509,40061569,40165023,42707611,40231869,40160333,40063476,1592529,40221406,40231900,40122124,19134119,1125113,35605650,40033406,42707772,35603627,40034968,42874030,42874043,42707615,40173102,42874033,40170041,40232769,40221199,40174867,40173059,19135521,40220500,19017651,40176075,42707613,40170064,42902840,40170034,19116027,40169948,35605648,40169897,40171474,42874036,19107233,40173094,46221482,42899095,42707624,40061841,19009040,40170066,40231905,40169307,42707603,42874041,40061664,40173113,40146012,1718702,1126566,42705513,42707758,40145585,42707628,40171482,19135545,1718960,40232728,1154147,19000944,45775133,40166391,19134112,40231924,40155525,40103241,40072109,40165015,40171444,40169976,42707786,42629510,1102527,19002435,40176088,19117382,40231866,40090254,44814192,40012264,40231907,42902888,40243114,40235694,19104083,19040310,40173156,46234297,19102016,19084973,19088903,40173071,40165050,40169889,19009038,40243410,40243118,40169901,40169909,40169917,42707617,19026495,19077250,19135551,35605860,40163872,19088398,40170063,40150023,40165721,42874034,42707369,40165009,45775132,42707779,40165016,40234428,40169340,35603598,19029403,40069473,40169956,40232766,40231871,40173116,40232721,40072153,35606063,40130481,40169873,40169994,40169921,40147574,1103351,40169905,19135575,40002747,19006108,19135621,19135515,40012595,40167707,40173470,40124834,40173131,42707610,40231806,45892404,40176063,40169911,40015856,42705499,43559979,1593601,40167705,40012604,19088907,40061559,42707785,40232722,19135628,40240172,19009037,40115863,1103361,40232764,1103360,40090249,19135629,40169942,40047773,19088393,1154316,19057739,40146011,40067519,40169337,40015855,40162167,42628938,19100176,19064598,40244087,1592361,19122001,19104301,40221151,40173130,40231815,1592528,40221203,42628927,35605647,40232724,40176061,42628937,40173133,19060338,40141908,46287478,40221220,19068246,19031907,42898524,46287287,42705318,40173088,42902993,40232706,1592530,19127010,19091639,42629505,1130585,40002748,40231859,40169862,40165027,40060797,40231816,40169954,40051251,40166389,40173086,40227598,42705484,42629509,35603601,42902864,19135535,35603600,40170047,40169983,40061394,40169940,40138577,40169982,40165046,40069474,40173107,40231872,40169318,40176080,42705464,40147572,40170002,40171397,40173128,40164989,40170016,40227593,40243106,40173073,19135531,40051254,40165712,35605848,19062895,40148383,44785556,40243109,40176062,19026459,40228398,40232700,40176035,1111319,40169336,19012830,40176059,40227595,40056819,40150538,40168605,19077075,40174865,40167715,40169344,40167706,19079504,40231964,42901816,19135522,19135536,40231911,40176089,40176069,1129105,19045402,40129972,40170009,40038378,40097622,19115675,40056060,40231876,40240175,40165054,40155944,40232770,40063474,40169882,35604844,19040309,42705306,40173471,40002779,40169985,19133261,40076970,42707776,40231868,19085669,19112162,44818053,19115676,1718697,40169329,40170474,40173134,19001066,40171438,19017126,40221223,1102596,1103314,40232701,40173099,42902739,40176074,42707789,35604838,40231873,35603940,40232760,19017558,40232747,1124957,40175255,40027797,40168641,40231810,19012855,42707761,40170058,1154313,40231883,40072457,42707755,1592525,19125147,40169308,19134268,1129233,1129232,40158128,40169880,40168596,46287331,40166401,19006029,40089950,42629512,40138913,40034817,19022055,40170054,40173101,40243113,19035020,40169814,40169886,19108216,40162165,19112635,40165715,42628920,40062122,40165053,40176087,40170026,40221213,40169922,40169325,19135568,35605653,40173068,40015851,40164995,1592507,40002756,19102014,45774946,19017165,35603634,40051250,40169978,42902645,40169946,42902413,42705516,40173126,40170024,40061845,44818383,1103359,40243194,40176093,40175251,1154184,19091877,42873586,40173078,35604997,35604820,42708971,40171478,1593602,19046078,40176064,40169949,40221208,42899097,19134117,1103401,35605659,40169960,40142581,19103711,40231898,19101416,40004070,40169933,1129234,42705322,40240173,42873360,40169988,40170065,19016660,19135561,42705315,40170018,40176066,35604998,40027796,40061537,40165011,40169961,35604842,19134114,40146007,42902935,40232738,35603631,40061548,40002735,40167714,35604819,40176077,42707777,40239282,40231867,40169926,40169965,40240846,40176065,46234296,35605644,40148811,40169996,40165711,1103422,42705454,1154105,40228549,40243107,19105691,40232754,40173138,42707732,40232741,40169992,19017128,42705503,1103346,40171480,40067520,40169869,1718956,42628939,40072156,19012858,19133845,46287468,40173127,1592360,40232744,40162915,19135516,46287481,40232726,42707625,40173124,40165026,40231808,40173098,1153670,19006093,40232756,40173469,40169324,42628924,35603806,40167708,40168606,40072151,40173070,40171445,40061149,19088400,42707602,35604846,40169885,40232732,40168599,40171396,40232767,40169316,1154183,40242499,42903251,42707798,35604821,19100097,40169319,19134121,40169945,40003543,40242655,19062853,40173103,40231812,40173144,19135016,42705303,40012598,19086605,40164997,44785557,19135532,19021994,40169959,40173096,40010981,42705494,19084235,40169890,40170665,40220499,40012266,40168600,42705481,35603633,19064406,19006107,42707780,40221196,42902889,35603936,19006057,40169894,40023607,19132884,1135449,40165002,46287467,42628929,40070565,40166394,19063890,40089957,40165057,35604845,19098900,40231908,35604839,46234114,42705483,40175249,40163260,40173142,1111353,40003546,46234295,40168592,40173010,42705309,40115862,40169958,40163869,40164992,40231818,19135572,40176033,40231878,40169343,19069615,1130589,42707781,40176073,40012254,40173135,42705312,40231874,40056815,1125264,40234430,40002746,40170664,40170013,40015853,35603636,19133293,1718287,19024384,42705462,19062309,40176058,46287724,40176124,19116026,40240170,44814285,40170037,1154186,35604999,19010299,40169314,35605843,42902867,1125034,40232752,40147576,40170036,19134113,19134047,42902727,1154150,40228548,42705307,1154146,40164996,19017163,40242495,40243733,40176091,40243117,40070563,40174853,35603579,1113755,19116405,19006027,40167265,40173123,40173093,19006096,40231813,40126417,19017907,19003010,40243120,40242497,40087855,45774941,40242496,40170082,19022593,40221214,19117381,40243416,42628942,35603945,19135571,19116519,1130617,19135509,40236566,19002649,40062121,40169947,40243110,40053091,40170057,40169929,40243425,19012829,40231945,35606453,42705468,40089956,40089951,1125112,42708739,19090528,19133296,40051247,19135579,19006097,42707630,19044949,35605172,40173767,40173132,40242511,40169870,40168593,35605176,45774943,40231966,1125232,40163870,19086214,40232719,40012615,40227590,40170051,40231917,35605859,40160227,19135570,40232736,1113237,40221202,40056814,40133154,40221195,1130616,40232735,19100815,40175247,40221230,40232771,40061545,40243413,40232569,1125153,42898236,40061535,1592359,40243419,42903081,1130615,40243098,1103353,40061541,40169315,40176090,19003054,40169975,44814284,40170666,19001065,19006026,42707629,40169913,40165006,19076368,42903372,1130613,40169927,19048381,40012257,40158761,42874029,44818051,40231865,1133228,1718700,40047772,40173104,35605649,35603626,42707778,40242503,40165028,40221232,40061571,40221233,42629511,40243111,42628923,40232757,40231943,40169997,1113580,40231965,40170070,40002850,42898522,1153666,40173087,40173080,35604840,40232731,40173095,40076968,42628936,46287333,1103343,40231931,1103348,42707775,40232703,40169196,40173089,40015181,1153668,35605837,40170039,40231903,40166409,42707768,40164994,40170028,42707767,40056820,19135624,19006095,40144437,35605842,1133201,42705502,19015999,40051253,40231935,40170017,19135528,40169970,40170053,40242508,40173105,19121194,40176060,40061561,19003130,42707727,1592526,40167709,1135465,42707619,19090529,40164003,40169875,19057737,40165049,19068247,46287725,40242498,19022417,40242504,40169987,19135627,40169331,40170059,40010980,40169968,19100219,35604832,40147577,1189437,19101052,40118177,40149759,19026296,40231923,40167704,40170007,40232733,40002787,40062120,19062756,42707770,42707720,40061840,40163874,40165022,40087856,46287476,40231909,35605174,40169304,40157251,40169963,19017560,40061570,40169999,40242500,42903170,40243123,42707604,42707795,40221225,1111165,42705455,40170050,40089959,40166407,35605840,19133298,40243099,40170044,40169971,40148558,40221209,19107328,40072154,46287474,19063636,19034292,40221227,40155168,40232748,40171479,42903391,40063127,44818382,35603628,40169962,40232723,1153704,19006109,40176078,35605835,40221218,42707607,1113581,46287475,42898237,40170010,40232708,40243412,44818381,40232729,1718691,35604836,19134009,40232745,40173106,19135538,40169898,40242512,40166388,40169937,40232710,40176083,42707790,19113603,19135529,40169972,40012616,42705323,19135548,45774949,42902544,40169998,40231811,40169928,40166405,40143348,42628922,40169305,35605858,40231877,40169984,40173765,19017559,40056761,40169872,40169895,40231926,40234433,1113752,1154281,40232718,40239278,19135549,40231861,19135560,1592531,42707747,40157106,40170032,40170029,40170048,40240174,42628930,19061990,42874035,1718701,40129004,19088399,40231817,40228551,40232704,42705311,40165000,42707994,19124325,42628943,40232727,40148023,40167713,40162166,40012262,40231805,40170005,40231858,19006110,40232768,42707612,45775134,42898523,40170071,40087854,40162169,42629506,1718958,1111320,40170076,40231863,45774945,1135448,42628921,35606452,40171476,42707750,40173009,40175250,40173108,40150024,40169910,40146006,42629504,19026494,40170078,1153665,40169313,19068249,40061562,19134124,40240171,19006098,40170049,1154279,40234427,40227597,40011280,42902404,40231936,40061357,40170080,42628934,40227596,42705482,42901173,40076974,19012857,19135524,40170003,40170079,19029477,42707788,44814191,1592355,19006025,40223140,19009036,1111166,40169887,19017908,40221221,40090248,40173139,40231899,1111352,40232568,40169912,19002985,19085706,35604835,40231880,40002786,1718955,1113562,40168603,40061666,42902803,40002849,1103398,40170004,40023612,40170000,40169995,19135525,42902560,19107321,19034973,19135552,40231814,42707609,19112160,19133297,40169860,42707796,40171443,19068250,40165021,1718959,19026493,19091578,35604834,35603941,19135534,42707792,40051246,19115674,19077249,19113600,1135464,1592362,40173145,42903067,42629508,42874040,19135514,40169341,40173473,40170030,40090255,1113573,40173118,19062350,40169939,19134049,40061546,40169899,19044981,19010025,1154148,40173763,40234434,40173129,42705313,42705326,40165008,40173152,40171475,19084971,19086483,19135692,40056817,1154185,35603625,40231807,19006055,40243108,19134123,40160458,1113753,40173069,40146008,40076965,40221395,42705504,40175246,1153733,40090253,19062493,40175254,19133108,40173090,40169907,40076973,40170031,40173112,40166408,19135765,40034818,40170006,40232750,40061539,40231901,40232762,35603942,19134111,40136244,40169874,40231915,42707797,40173153,42705324,40243119,1133224,19134116,40165029,19057738,40061547,40232737,42707724,40169470,40221204,42903132,19078591,40169918,40136564,42705304,40231920)

In [ ]:
statins=(19017058,40174529,19125465,40116626,1539463,1592148,19129526,42707427,19129329,1592142,40165245,19129171,1332496,40225105,42901620,40165244,40165258,19003030,19031783,19087540,19134900,42901621,42707430,40078418,1545992,40165637,40165261,40083921,19128153,40165644,40175403,1592176,42629542,19068781,43527031,40225098,40174526,40165649,42901618,40000954,1332522,1332500,1586231,1518133,40168499,19089429,42706212,1539462,40175397,40174530,19134899,40243622,40008941,40165636,1539464,19128149,19087538,19023384,40165648,1526479,40165262,40243632,40175395,40165248,19071065,40175391,19134889,1539468,1545994,19022104,1545998,40083920,40095248,1551932,19103503,1592119,40165246,40174532,19004082,40165641,43527040,40174521,43527032,42706209,1586229,40147344,40174525,19112569,40165263,19038968,19023563,42706204,40243631,40012612,19086163,19134901,1545999,40000953,40165249,19023409,1332497,1592117,19023383,42901623,19128148,43527029,19027242,19102497,1592112,40243628,19134897,19128164,19134742,40243626,40175398,1539466,19129330,19019116,19121752,42708401,1592149,19031851,40165260,40175390,19128168,1518135,19122208,19120294,19123593,40165252,40092964,19040307,40175404,19102730,19075051,19023405,40243633,40175399,19122209,42707422,1518136,19084874,19129170,19134783,40165643,1539502,1518134,40174522,1592180,40144458,1545961,40058433,40174524,40175393,44506641,19129801,40000951,19122210,40165254,19017056,1539406,42707423,40137749,19128157,19123592,40175401,44506452,1586260,19003051,40148606,1545997,19134931,40224130,19102566,1539407,1586230,1592085,19022088,42707426,40225102,40243627,19134896,1592118,1332494,40165646,19128156,19128155,40165251,19129173,44506638,1592121,19134932,1526480,19128160,42900899,19055967,19085787,40223188,19022087,19134743,1551899,19019115,43527035,1332499,40165647,19128169,19134930,40175405,40095246,1592143,40058431,40225104,19075040,1592116,42707416,19124977,40175389,19124976,42706199,1332495,40054620,40058434,1586255,42629541,40000952,1586256,40165253,19121753,19071064,1549722,19128152,42708398,19134895,40225101,19031782,19128161,40223186,40165645,19027244,40165639,40225103,40175394,19077497,19031830,19125466,1539411,19128163,40223187,1113565,40099653,1592175,19019117,42707431,40224131,19113907,40224128,40165256,40175402,40224129,19127316,40175400,19023385,40078420,19128162,1545958,19134825,40175392,40165640,43527033,1539410,1539465,19077499,40225100,1551898,19023382,42900902,43527037,1539467,40175396,1592145,19128151,19071067,19099943,19027248,19075052,40165257,19128158,43527038,19128167,40165247,1592115,19102499,19038970,1592150,19128150,1510813,1549723,40142114,19128154,1592174,40000917,40165259,19077498,19031784,40243630,19037852,40149393,19017057,19116988,40174527,42708403,1549686,40175406,19102567,42708405,1592113,19038967,40174528,19117064,43527036,1592146,42707417,43527030,19128166,1539469,19055966,19134741,19084875,1545996,1551860,40165642,40099654,40092963,43527039,19121751,40165250,1592114,40174523,42629543,19102498,19055969,40243629,1539403,40243624,42901619,40165650,19087539,42706208,42707418,40174531,42901622,43527027,1545959,19128165,1592147,1592120,19129172,19004081,42705213,19055968,19128159,43527028,42900896,40243634,43527034,19129425,40165638,40165255,40095247)

In [ ]:
msknaids=(45892518,40054686,19105627,19008011,19101638,42799742,19008096,19008154,40059331,40048836,40240118,40102519,19007792,19008070,19008094,19106554,19024596,43011764,19008130,40131970,40054141,40064722,43525702,40241474,19024622,19134398,19116350,19033854,19095707,40032109,40160449,19010772,40055254,40162361,19007988,19098339,40162344,1395655,43531821,40083341,19042157,40227877,19007985,19008123,19056874,40240602,40099354,19103555,19081236,1189776,40237531,19008181,1153969,1360337,35605793,19129422,42902226,40038015,1118088,19007909,19107414,40243192,19122172,19110135,40162159,1360379,19115531,19018046,19019272,19105162,19109952,19009665,19007821,1115066,19008064,19029394,19062830,40048771,1124300,1150373,40148513,1150348,40240114,40099356,40112435,19007906,1360433,46234375,19033853,19104780,19081524,19102158,40131393,19024626,43012642,1395618,1389810,40162331,19001950,40100422,19033923,36249255,19132864,19041246,19024660,19116278,19033925,19109675,1395616,19062730,19024599,19126142,40064724,1036641,40102523,40099080,19064902,19081052,40050539,40041352,19078521,19109871,1189761,40111803,19110711,40048783,19116356,1197755,19024659,35605791,40162328,19102082,40125597,40149696,40032123,1360381,19001979,19033986,19124745,19109309,19029441,19108607,40064742,19081525,19008066,1185927,40165703,19090667,19019665,19010434,19033856,40162341,1115071,46287423,19007730,1118047,19007863,19012743,19052755,42707759,19135494,40057436,19071691,44507672,40072979,40099083,19062851,40149047,19111552,19024537,42898961,19008021,19117118,19090497,40166812,40125368,40234338,19008189,40056064,1135710,40054693,19071234,19110569,42903335,19010234,19008151,44785233,19039707,19097794,44815866,43525438,1592915,19033852,40169658,19132995,40026487,19033823,19080098,19054229,19008129,19022488,19007989,1185930,19024593,40097557,19016199,19104888,19024591,1395617,40037724,19008069,1103376,19007728,1189755,19003022,19054527,40032414,19017191,19106418,19007826,1395599,1236615,44507670,19102108,1185931,19016761,40241655,19029398,19008061,19086910,1178663,19044792,40234335,19088916,40237054,19129266,19116281,19008126,19008100,1360436,19002860,1197740,40239169,19106423,1178727,40083340,40054144,40054158,19024598,42902224,19055715,19029025,19021282,19019273,46221593,40162325,19045660,36249249,19122680,19034076,40102009,1150349,40239208,1178668,40234337,1189754,40242726,19007862,1395619,40102522,19007725,40072972,40050535,19008026,40026440,19116279,40099355,19008025,19016770,1389858,46234376,1189777,35604404,40055252,43532361,19135737,1113673,19008018,19008019,19019666,19001976,19022024,40165704,19008186,1395641,19107641,19033922,42903325,40054608,40162304,19044794,40010473,19033921,19105164,42705489,1395654,19072152,19044793,19033893,1153967,19039706,1113648,40044520,1395633,19008062,19066363,19007947,40162334,40054692,19081499,19007824,40057838,40010475,19033928,42708549,19129914,1185922,40050244,19024630,40223093,40243193,35605790,19110913,19123499,19007823,19110944,19010679,40054689,19135740,19010578,40242479,36249250,19024539,19010680,40102008,19105867,19024538,1592916,40099353,19062852,19135648,1156419,19110567,44815865,1395592,1360341,19033892,19116355,19003591,19069191,19021283,40237540,42800848,19015993,19105570,40242478,1124637,40102363,19124629,19101848,40041350,40132232,19100505,19008183,40057841,1177770,1236609,19104691,40031159,19007990,40058303,19097732,19016791,19132863,40041025,1153928,19081237,19029400,40099081,40229836,1135735,1395598,1177843,1150345,19008024,19033527,1197741,1118084,19124388,42800830,19124278,40167593,19007861,19007907,19056645,19071933,40239446,46287318,19103398,19129314,19024651,19034020,40162337,19033926,40223075,19007942,19033924,19007910,19066369,42705485,19024594,19024625,19010435,42800833,19002990,19033827,19065658,1186024,19033719,1360437,44814230,19007770,40041037,19080243,19078461,19109308,19024629,19116476,19112207,40220405,40162156,40235700,42708329,19121393,19103287,40238640,19019074,1360409,40234336,46234377,19016267,40236810,19039704,19133333,19110912,19034015,19087095,40057837,40229832,40055251,19008188,19102047,19081495,44506493,43532911,19093339,19118682,1360340,40239448,1395653,19078522,19104055,36249253,19107783,40162335,19082874,1177827,1395634,19013337,1177769,19007794,19102073,40055253,19116353,19007766,19010408,19054448,1150347,40057836,42629124,19099560,40077961,40131392,40099082,19007827,19008187,19007763,43525439,19116277,40149311,19033828,40238638,19008158,40128023,19116255,19134910,40097542,42800831,1036639,40237400,1177822,40041351,19115710,19024652,40242475,40129525,35605797,1236616,19044795,19111123,19065929,19113508,19007796,19001978,40102010,40237057,1189772,19017902,40026486,40163128,40056408,19109947,19049813,1115126,1118087,40064710,1395573,44506660,19010266,1360411,19008095,19024592,19007825,1178666,1146810,19134640,19135647,19011355,19121479,1103374,19016200,40133841,1389856,19101640,40010476,40032417,42705300,19010748,19023335,19127947,19062230,40173035,19007984,40148508,19005999,1118045,40131971,40238639,1395597,40166811,1113672,19126960,40102364,40057839,19029329,19133853,40082937,19106211,19011359,40162307,36249254,1236613,40162305,40237056,1178758,43011763,19010771,19007829,19129265,40099085,19103697,19024600,1360440,19113507,40162351,1153929,19008092,42902402,19086938,40132234,19007904,1189778,19095703,42707583,42903238,40173034,40092758,19007987,40237539,44816218,19081496,19010407,19104613,1236611,19017368,19034080,40242480,1395581,19001972,19110557,40102518,19023037,19001974,42705487,19023481,749964,19008022,19010083,19110133,19133331,19033891,19008097,40164850,1177480,19135542,40242729,19003570,1153966,19071692,19005998,19105576,40167595,1115008,1115093,19136654,19008159,1395620,19086517,19021884,40101587,19010405,19010235,19007867,19081235,1195492,19125902,19033927,40237055,40111096,19116354,19024653,19007795,19008125,40097555,19024595,19081053,40050233,1189775,1150351,19007828,19007870,1395621,19063311,19001980,19054173,19109310,19103639,43532886,40029837,40054148,19007981,43013637,1395593,40162353,19133852,19077246,19103930,19067480,40059388,19034984,19018047,40239173,19008020,42799743,1195536,19066364,19001975,46234389,19034048,1186053,40237060,40041349,45892517,44507671,19111125,19078440,19008028,1195512,19134399,19110942,19007769,19008128,1115092,40054138,19106210,19100506,1177740,19029393,1156378,40102105,40162360,19102018,19126681,42800827,19054654,19024623,19112951,19113500,1236610,19026060,19007950,1189779,43560518,19033824,1195498,40162359,40167592,19100503,1189757,19078754,19106827,19007983,19104834,1146847,19041245,1115125,40077954,42800834,19007982,19125665,43013636,1153968,42898962,40038017,40057787,19029327,40044561,19113499,40175618,1395612,1036636,40131490,40097558,19109951,43532694,46221591,19011358,19008098,19081233,40237059,19007903,19054447,19095708,40237532,1395615,19009664,19086937,19007943,40229831,44785231,19045701,40093264,19104887,40227878,19117011,40026439,19122173,19017369,19033855,40239172,40131388,19063280,1360372,19088915,19038550,19008121,40077949,40097538,19116280,40010474,19008156,19106212,19007791,19088021,40229828,19007764,40053099,19007822,1395580,19080097,40054694,19106277,19008030,19100504,19118323,19024627,19017085,40162308,19132998,19102011,19121480,19086931,1186022,19016543,40167594,42902225,19034077,40149048,19039703,1360333,19007830,19008093,40044542,19088022,40038013,19104772,19115997,40041348,1395632,19077214,19019073,19029328,19063002,19007726,35605799,43531820,1177801,19088906,43012139,19008013,19024658,40131967,19010406,19099823,40026438,19081234,19108132,19065622,788038,1146943,40238641,19038508,40161250,19110134,40099084,19049814,40239447,40032413,19033896,40044537,40057840,40167076,1103375,19087999,42707592,19022250,19108553,40102524,19008157,19111124,1036640,40162330,40241657,40026459,19016997,40235198,42706379,1150372,40239167,40229830,19135739,19009669,40162336,44506494,19116276,19101779,19102263,40022480,19042155,19103397,40239450,44785229,19135664,40237058,19123153,19008099,19101847,40237399,46287424,19086518,19024621,19135493,40244387,19001971,1395601,1150346,1360402,35604405,19118236,43532891,19100579,19105252,1118091,19101639,40227876,19007944,19008160,40028644,19081526,19081828,35604406,1389834,40044538,19134039,19008124,19033894,19047955,1197739,1115070,19007869,40083339,40012236,19007941,19029024,19010749,40162324,19098338,40239445,19113321,19023192,42800828,19012742,19024628,19054057,19019072,1360380,42902154,1115067,35605795,40053093,19068308,40167591,40133434,1124636,19101870,19008029,19010841,19103379,1360432,19008016,40092942,19101674,42903106,19010810,40044543,40050525,40239170,1113674,19132997,19023556,19100502,1360441,19007864,40160485,46234390,42902222,40239171,40131968,35605798,19095238,19008023,40239449,19008068,1118113,19112675,19024751,19108432,40164842,40055249,19108608,19095274,1195496,40242476,19049709,19008017,19041220,35605794,19111551,19024656,19007765,42705297,19021918,19033895,1360373,1186000,43013638,19041244,19008015,19002983,19077212,40236811,19009668,19110132,19107293,40102365,40162306,19108710,42898957,40162158,40243947,19039705,19002982,40041347,19105078,40111802,19077213,46221594,19007949,19086933,40242477,40148512,19008184,40054688,19024657,19129205,40229829,19096676,1153930,19001977,19099551,19034019,19007948,40038016,40240590,19019071,43013639,19116254,40125598,40098296,40239168,40240601,19126256,42799744,1189773,19081500,1146916,40048791,19007797,40162346,1360378,1389857,19099780,19104949,1189756,19122679,19008122,1197754,1197736,19118237,42800829,40041024,40162347,40082936,40229833,40029836,19097733,19007761,19110131,44785230,1360405,19054174,42708328,19066130,19110566,40078256,40131969,19134688,42800832,40241473,1360332,40038018,19008152,19008127,35605796,19105553,19086932,19015997,19008185,19086934,44785232,19033851,40082938,40162157,40038012,19122492,19088000,42707760,19129206,19008155,40233631,19107142,19007908,1135738,19033825,42708550,1189760,19016898,40057842,40237398,1395594,19007946,19034016,19086516,40166813,19106828,19087998,19024624,40097554,19019050,1395596,19106273,40044539,19007945,19011356,40242727,19104892,19011360,40100421,40165705,40131780,19007727,1136980,19018431,40131389,19026061,1118048,19022893,1360336,19123299,40059389,19003592,19024540,19008027,19024597,19113322,19112950,40041036,1236614,19087094,42707582,40044562,19010540,19034478,19021712,19007901,19024655,19116916,19081527,40126120,1395613,40235197,40038014,1036638,19081497,19112206,40234334,42705490,19016762,40059332,19132996,1185999,40055250,19024654,19102074,1178665,19116915,40242728,19007902,40165702,1236607,46234388,19121394,19068411,42708585,19001973,1146915,40097556,1195497)

In [ ]:
oralhypo=(1525215,40164904,43526474,19079465,40139022,19006931,19099409,1718913,46287690,45774534,19125040,40164897,19045624,40229048,40220861,40046212,45774751,42708078,44816276,19060469,40165975,1525242,40100066,40168371,1597760,1597798,1718914,1530020,1503297,1718905,43013924,19079990,40163929,45774893,19081296,40164884,40165689,19030579,19063991,19117691,45775868,19106704,40164928,43013896,44815996,40139098,19030576,44816086,1597776,19088111,43013884,40220858,40134119,19099408,1592801,45775620,1594976,19135385,45774721,1525244,19035533,1517998,40058199,1529359,45775861,43013913,1560171,45775456,40170911,19102035,43013909,36249800,1592799,19085096,19125047,45774716,40231396,40164943,40164900,45774894,19059800,1515250,40164907,40030457,40165973,40046145,19106977,43013919,44816275,43013893,1718906,40165685,40004923,19025947,45774440,42705842,42708174,46287680,40164911,40164948,42708172,40239219,40167191,19077638,19088818,42705990,19126618,44785829,19082032,19027259,43011499,19006933,19077681,1515249,36249793,46287679,40231392,1559685,40136769,42708166,40135481,1516794,19069554,40170913,40166816,40164933,40168369,40100068,19109284,45775860,42705859,19024507,45774443,40166045,40164908,19087996,44816283,19110957,40163928,46287684,36249799,19092911,46234111,19125041,1718912,19125050,40164940,42901620,19121940,40164910,1718909,19100593,19106976,1597756,42901621,40165687,46287687,19001410,19030572,19083254,40165996,19122366,19065480,1516795,1594980,40089088,19106975,44785831,1559684,40176046,40165994,19037213,46234470,40164889,1592317,19030445,1529360,1502873,19079293,19107329,40220371,19125222,40046204,19021786,40164886,1559784,19024504,19117171,45892174,1529358,19027257,40098705,19107110,40220860,40054705,1592805,44816049,42901618,19080793,40220863,40168370,1718712,1592319,40173236,19125044,40164921,19025944,1592802,43013888,43011498,19113780,45774708,45775863,19116145,1592804,42708087,40171450,40220770,43013908,19021787,40243622,19033910,40160874,40164883,19001444,40229049,1592803,40164923,40046201,19030575,1502817,40054708,40231403,19045606,46234122,19047612,19107508,40153147,46275423,40243632,1597795,19085097,45892179,40165993,1529352,45776000,19021312,1502818,19035535,19024505,19017854,19117418,19099208,44785477,40069900,19120529,19117693,43013905,40003412,40164891,19067200,40166039,43013900,1718901,43013902,43526471,19081705,46234119,45774717,46276271,40164894,19082949,46287691,19122368,40166041,43526468,40135391,40164937,40163923,1502811,40173239,40164892,40146999,19081914,36249794,19099314,40173226,19077682,40164920,1502810,19134624,19048692,44785834,19117416,19023063,19099245,40163926,19099209,19060491,1597773,19022851,19081706,40164929,40164905,1516766,42708167,40164926,19106860,1583728,40166815,19006907,1592657,1718604,40030438,45774449,1592652,42901622,40030439,40046211,40239218,19125048,1718910,40163925,40159908,40143966,19036967,19078028,19107077,19115458,46234121,43013906,19030443,43013927,40164924,19135404,1592800,44506520,40164901,40056706,40166038,1592655,40153151,19024506,19115459,40165995,1516770,19048032,44785473,40063357,19054444,19017144,19100294,19045626,40239220,45775859,19017142,19116146,40164912,45774722,40243631,40166044,44816282,1530019,1547505,45774720,19045630,43013929,40164902,40239221,43013915,43013923,1525217,45775999,40099632,40168375,1503328,1510202,42708086,46234110,19054443,19071495,45774435,43013910,46287686,42708091,40229047,19017856,43526466,19033931,1547507,42901623,19045625,1547508,1515258,40164896,1560233,19029061,19125051,40243628,1510208,1559785,19018784,42708171,19125218,36249790,40164919,45774445,43013904,40243626,43013885,1597796,40099897,42902527,44785832,40129534,43526465,40089115,19059796,19083253,40046207,19083696,40239222,1510206,1718599,40164925,45775864,19117424,1718915,40167636,19088819,40164932,19125562,36249791,43013917,19117419,42708176,1597757,19106974,43526473,43013895,1592797,40117725,40164916,40164888,19135388,40164947,1718706,40044221,43013898,40044254,40220852,1592320,40166043,19125223,19135387,19107111,1597759,46287683,40164913,1559782,43013921,19060409,19113845,19023585,1594973,40098707,1503327,19047663,19048034,19125046,42705866,40056705,40046202,40231394,1525221,19129178,19048693,19036966,19125219,44785833,42902992,19135386,1547506,40089089,46287688,40164898,19125042,19112824,1718601,1718709,43526467,42705862,40164906,19045628,40165974,19099207,43013897,40221118,44785830,42705857,45775866,1515251,19024510,1718900,1592793,19110155,40167634,40165972,40220864,19088427,19030573,40166035,19117420,19040970,1597761,19117417,1559729,43013911,1718908,40220855,40164903,42902689,42708168,40046206,40044252,19047638,42708177,45774710,45774756,40220373,1597775,1718898,19001408,19134642,40167189,42705839,40173237,36249795,46234120,42900902,40239217,1718703,40220862,19030580,1718902,40167193,19033909,46234112,40229050,1597799,1718708,43013887,40044251,40164914,19125043,1000979,19099278,19047828,45774712,42705992,45774709,40165970,45774714,45775624,45774753,40166814,40126689,45775619,43013928,46275422,40030456,1718911,19018043,19001443,19079986,44816093,1592798,40168368,45775867,1510207,40231393,19103625,19107377,40220851,40164934,40063355,40164930,40044253,40220376,42902983,42708090,40229045,45775862,19077659,19033932,19102037,19129176,19099070,1592323,40010765,40173234,19082950,45774723,45892178,19109211,19045605,19067365,40171449,40164917,19045629,40167192,40166036,45892172,1718602,40168376,1502829,19112585,40167637,19110190,42705988,40164945,40243630,1502827,40164939,1718711,40059670,19085004,44816284,45892177,40089090,40164927,40164887,40165686,45775623,1502916,19091661,43013920,40164935,1559760,19035534,42708169,19120528,40063354,40164881,1597781,1597774,19059797,1515254,19117170,1583722,19101729,40173227,19088817,40165998,40099898,44816332,46234471,40165688,1718596,43013912,19107661,46276272,40058198,19125045,19099313,40046203,1529331,43013925,1525243,19117200,19107624,44815995,19078029,19120257,19006932,40167635,40143322,19054445,1525216,40127005,43013901,40231404,1547554,40126695,40231387,45774447,42903341,45775454,40163922,19067231,46287409,19117199,19129177,40220375,40096453,19120562,46234118,1594978,42708089,1502855,36249792,19117169,43013930,40164880,1718903,42708088,1718598,40171448,40173235,19099055,44506518,46287685,19110191,19006910,1597758,40063356,40164885,40166040,40166004,40164938,40166003,46287678,40108537,45775455,40164895,19001441,1597772,1547504,1547510,1515253,43526475,40164918,19126619,40165977,43013903,1597801,19106705,40098706,45892687,45774752,42901619,1502830,45775622,40243624,19024503,40164931,40243629,19077684,19001442,19125049,45774758,19116148,19103441,19067232,43526470,45774713,19122137,42900896,40239216,40243634,19113846,19047829,43526472,1525218,19029030,19117422,45774724,19106521,43013922,40164941,40163924,40136854,1515256,19023426,19110958,40117785,42708173,19122367,40166042,19023424,45774450,36249798,19036965,40164936,43011831,40148129,40220854,19110194,1592795,19117423,1592796,42902866,19120166,45892175,40166002,40167638,19017855,19006908,40243633,44506754,40171451,19116147,40165692,1502816,1580747,42903071,40165969,45774535,43013918,46287689,19040969,40165997,1559728,19134639,40231388,1559761,40229046,1592316,40003413,40063306,19025946,40231395,19077636,43013899,40004924,19117198,46287408,19099246,44816285,45892176,19097821,40044222,36249796,46234116,1718705,19086484,1502812,1718907,40166037,1718713,40164899,40136770,46234117,19117692,1502819,40134120,1592806,44816273,19100559,45774895,19112584,40164922,1502856,40165971,42903382,1516771,19110153,40164909,19024508,1525220,40054308,45775865,40173238,19117945,44816094,45774719,19024502,1592653,19018783,43011830,1594975,19047414,40167190,46287410,19077637,46287682,44816274,19006909,40099633,1592794,40153895,19125221,1594979,19024509,40220857,19060470,40059666,19109283,19081704,45774718,40164893,40166001,40231402,42708170,19001409,1502809,1592654,40163927,45774757,40100067,40168374,40170912,19120530,19088820,40044224,1530014,40046210,19023425,40164946,1718605,45774754,43013916,40054707,45892684,43013907,1502826,40078056,19117421,19125220,40153892,45775388,43526469,19088426,45774444,42705865,19082961,40164882,40164915,1597792,19110154,43013932,19078030,42708079,40046205,40231391,45774715,40054309,40046209,40059665,19029029,1559786,40165976,19069594,42708175,40243627,40022442,42705856,43013914,43013931,19067366,40059651,40063353,1718904,19081295,1592656,45774759,1502913,19048036,45774436,40164944,40046143,1718606,40063352,40220859,40164942,43013926,19120563,40164890,19120648,19129179,19033498,1718899,44816083,45892171,40063350,36249797,45775621,42900899,40054706,19091157)

In [ ]:
painnsaids=(1718643,19009296,1125358,42799084,44814282,43012471,19118873,19030206,40166843,1592595,40231884,40235313,40005722,40231927,19071607,40231934,40223229,19054293,43526532,35605899,40002652,40162564,40231862,19125286,42708870,1128013,40223246,19118898,19044375,43012477,19111627,40133153,42800254,40235351,40139826,40241556,19030386,1126983,42707871,40162525,43526536,44817952,45892777,19107791,40241593,1127685,40002741,19037350,1128461,40238702,42799137,40162494,19015603,40162490,19106547,42873700,40002772,19118876,42707443,1164262,40235363,40162537,40162489,40004040,42873434,40096471,19118529,19135523,19072467,19041290,42708990,40231902,40012267,42709059,40004043,42707889,40233286,19073406,906890,1126458,42706760,40002680,40235355,40128197,40162480,40231879,43013330,40151317,40133431,40002803,40229088,42705385,1127642,40139319,40002733,40221915,40229065,40005668,19080555,40223224,19121452,40227636,19041382,40003571,40162593,46287285,40233252,19118870,19116275,19106609,1137228,40240313,40221973,19061733,40142618,19044380,40099992,19041427,46234324,19125402,43560443,40162610,19118941,42706849,40221967,43532896,40156651,40231885,19076395,40005633,1127782,43011745,40221900,40005700,40005696,42629297,40142584,40241561,40162533,43560454,42873633,1127952,44818202,44506379,40235306,19041618,40223244,42707812,40033571,45892529,40004098,40140519,40004020,19118878,40124837,19041196,19072343,19107288,1592488,42901547,19110305,40237581,19135550,40221912,40236704,19072180,1112923,19084050,40138388,40124835,19073403,40241558,40238706,19125628,19041466,1127045,40243248,19128412,40003551,1135360,43012486,40231910,40031896,40221949,19119716,40238731,46275807,43012462,19073436,19009295,1115064,19084136,40004119,40155107,1127784,45775855,19115391,19024358,40229070,43012457,40148365,19126654,43013280,19118877,19046167,19119718,35606326,40002751,19026059,19041327,19073405,40240319,40166019,40003685,1126534,40233282,19020053,19063316,40128278,19037344,1127020,1127897,40133164,44814283,40231944,40162478,43526545,40130208,40231864,40229104,19069642,40011363,19059093,40242531,45774828,42901487,40003545,42898301,40235317,40233260,40142582,40223249,19105591,19111634,40237561,42873426,19071886,42799095,40002695,19041254,19041324,40236709,35606610,43013288,40002750,40221916,40152853,42800378,1594739,40236691,1127021,42800368,19135128,40142615,19123144,40003583,19015602,19092792,1126910,40221965,40002775,40005764,19062859,40002810,42898297,40221930,19118892,19072237,19115389,19041360,19118530,40243253,19123625,40238720,44785084,1139700,19072177,19041430,42800361,1127576,40223208,19042793,40162473,40237393,19107439,19121388,40233259,19105498,19088814,1127790,40235305,40099990,35606612,19122502,19041620,19060003,40103234,40004034,40003615,46275457,40227615,40005795,1127441,19105499,44815751,1137524,40004036,40231925,40103233,40155170,42901950,19129549,40235328,19080480,19118935,40162536,40238708,42873315,1128019,40005743,40103277,40002729,40003691,45775633,40221910,44814281,40162562,44818015,1593434,40004071,19107518,40103276,19041390,40162517,40005648,19125577,40005704,19028541,40229121,40162498,40162512,1130614,19041658,40162487,40223203,19016438,40002629,19118889,40231922,40162613,19041659,40004019,19028607,43560444,42707622,44816257,40137102,40235356,40002870,19042794,19072205,40162638,19072316,19072432,40002872,19043334,35605900,40133428,43532536,1177664,19041351,40162602,19086578,1127900,1128796,42799586,19042827,40227629,40231881,40004102,40231913,40162591,19123718,40223242,19021078,19103532,40231896,40155849,19053850,40231933,1112807,1127812,40133433,19016516,42706850,42705649,19127535,40238727,45776995,19132641,40003557,40100000,40233323,19041614,19037342,44506588,40227601,43531896,40239568,19005065,40221011,45776409,44506706,40162465,40162598,40002742,40220904,19073367,42707525,1593664,40156649,1127105,1164222,1100144,19135131,40229146,40162485,19065215,40221958,19086544,19107245,40004029,40238732,19041619,19062156,40003529,19016042,40002641,1195352,19107242,40004115,40231918,19118920,1126960,40139763,42708154,40169408,19030061,40162493,40004042,1194105,19108622,40237394,40128203,19124722,40236685,40239589,40002755,40233327,1127683,40153520,35606616,40242340,19009294,42873511,19067051,19105502,40002858,40162543,40241560,40237573,40084352,40012277,19119249,45892195,19059506,40128202,40227610,1126875,40243234,40241559,40005699,40227634,1593051,19041385,42873425,40139318,40239617,19042684,43013203,40227626,1164310,40221954,42708351,19122787,36249120,35606607,19102750,19118927,1129030,42799194,1126742,40231914,40005665,43013468,1127495,40153556,1126873,42801302,40233281,40002706,1127497,40002655,19131644,1128797,19121933,42800365,19042764,19041468,40231929,19105493,43013289,40124796,40227608,42800280,19042730,1164309,19115995,19118884,40137452,40229139,42799140,19061451,40235387,40233267,40002640,1195380,40003480,40005666,40002776,42873543,19078895,42705667,19042763,45775060,40162578,45892196,40005727,19041469,40223210,1126496,40142590,40002864,40003645,19129632,19073437,40002687,40145220,19108460,19107258,40235359,40099999,19122788,40160964,40135542,40005681,19105563,40150343,40002780,40003630,40002597,1127494,40235373,964639,19073811,42707623,43526543,1592599,19122786,40225816,40237584,42901151,43013327,46276241,19118859,19124627,19119397,40243236,40003552,40162586,40005701,44817981,19113801,1164108,19125056,19123145,42705669,40231908,19119725,1140361,1140358,40162479,40003566,19129556,19127033,19103708,19119396,19122378,19085666,19073471,40233275,40162548,19093132,40148372,40162538,1593663,40223197,42873370,40005686,40162500,40237567,1164267,1126748,19109782,1164271,19124735,19071255,19041506,19124719,40241549,43013338,40003546,19132080,43012470,46233707,40149732,42799195,40221903,42705802,1126961,19131368,43531887,930634,19107289,1164305,40240385,35604564,40065710,19133768,42873196,45892197,1127691,40135184,40162472,19105530,40033573,40002865,40236697,42707781,42902183,1127436,19027499,40229123,40004090,36249946,1127078,19073435,1112839,19041870,40003481,40231874,43559972,42629299,1592492,19042824,40231878,19118900,40162511,40099997,42708436,40162486,19133293,40003568,42873593,19005996,40002588,40238703,19041329,19118897,40002672,40003611,40003676,19077204,1128015,42902252,40002746,36249549,1593430,1164227,43532884,40142595,19113071,44506604,19041504,1193724,19088349,19073407,1125392,19041198,43532687,40003489,44816115,19028034,40236687,19062698,19029146,1194157,44814285,42873701,43012129,40233319,42708001,19054297,1164223,40124830,1127044,40225817,19107554,19073362,1124009,19110001,19103486,1134771,19119395,19112656,19041617,19118885,19113800,40017312,19128785,44816288,40002840,40004106,46287728,19118866,40034241,40221976,40236680,40003558,40240414,40003528,40238721,40004022,19118847,1593753,40004097,19034417,1125357,46234326,40002697,19119723,19070139,46233709,19073370,1127898,35603795,19071214,40003506,19041656,40162477,43013308,40002666,1156937,19118924,19101846,19080556,43012452,40235354,19076692,40135188,40003576,40220877,19110239,1193364,42707787,19073434,40223196,40124836,40002794,1194792,42706759,40009893,40003614,40100001,1128053,40005781,40239413,40223193,19027831,42873437,40002846,42709001,43012080,1718639,43531905,40235357,40002635,19059539,40227620,19072207,19037381,40002832,19111131,19042768,19019815,19060889,35606325,19119721,965069,19119394,19106598,40241539,19105367,42799157,40162597,43526547,40003567,40220888,42901494,1125393,19111630,40004113,40240305,19005997,40162571,43560439,1592198,44818282,43560447,19072173,45892551,42873702,42708011,19135529,40231877,40002766,40002797,44816263,19107557,40235304,19082148,40002867,19092086,40002873,19123684,1194158,43532899,40103363,40102696,40239591,40162569,40239583,19046504,1127524,40003701,19128411,42705323,19037343,19041325,40229097,1126501,19115994,19029324,19119251,40231903,40002586,40162515,40162559,40004117,19128784,19058753,40145251,1153668,19105404,19107323,19065578,42708917,1139701,1594740,40162506,40223255,1127527,40233271,40005659,40139753,1127133,40231863,40140520,19095919,40236710,1128055,40221119,19118890,19072157,19075752,40002745,42799110,19072236,36249122,40231858,40004041,40238717,19118902,43013341,40003709,45776385,19087813,40231897,1126459,19120283,40237570,42901538,40161012,19111114,19005260,1126532,42708031,40002708,40221921,40134214,19026875,19105665,42800358,40162608,40231936,1140385,40227597,19119243,40002771,40235389,40235377,40162503,40003692,40003507,40235376,19072203,40223207,42901532,40238729,1127989,40004051,40162467,19128786,40169409,19028010,19105503,40005780,1120327,19129545,19118926,19041381,40004091,1126954,40220875,45775668,19065576,19062342,42873435,44817996,40175529,40003632,19042762,19116434,19092765,40238711,42799154,40162528,40002699,42902248,19054935,40127049,19106759,42707780,40240315,19009297,40236695,19043333,36249126,40136240,40002618,19123272,19126695,43012494,19045935,1126562,40099967,42901802,19103360,40002757,42708916,19108637,45775022,19084568,1126540,43532514,19075776,40135536,42902186,40221947,44817992,19111638,42706837,40237580,40162523,40229107,40227633,19115390,42799113,40162481,36249548,43013317,43560023,40003503,40221956,40002696,40161010,1125321,19005992,42707523,19042795,40162554,19106658,19044379,40235340,19090578,19107574,19126753,40227596,964637,40070844,40221944,40148383,40103376,40227606,19017986,40229071,40005754,40162483,40235386,40223239,40227595,19015607,40056582,40003613,40136245,1126500,40241553,40162541,1593431,40002743,40235327,40005667,43532827,40002684,19057665,1172211,19065152,44785556,40237572,40169439,40155831,19135205,40139756,19118938,45892550,45775743,19105409,1100451,40235369,40221911,1125315,19041422,40162504,44817983,42901548,40002779,40227604,1127104,19018933,40103368,19127034,1137523,40002654,40017325,19073521,19116574,40231868,1718640,19086543,40002736,40004031,40162513,40139751,44818053,1128798,42707528,40058009,19079504,43532956,40241572,19059060,19042337,1127647,19117380,19111646,19113072,40229128,40005777,40223192,19131369,40233294,19119849,40002764,40133163,19041510,40229083,40236712,19107122,40002822,19121423,40004016,43526535,19055711,40003479,1128794,45892545,19107266,40162499,1113262,1129105,43012458,45892403,19045402,19042335,19098457,19054409,40059885,42898295,19010265,40229089,19135522,40231911,19115122,36249123,40005810,40231876,40235308,40003523,19119714,1137525,40103223,19108619,19135281,1126881,19042796,43532823,40238722,40002856,40221971,40004074,40155944,40004116,954856,40012273,40003547,40003675,44816256,44785548,19017402,40004056,36249872,40229126,40003526,42902182,19112655,42799133,40005778,40231937,19047813,46275563,40223245,19080856,19041197,44507436,40002798,19048847,40002710,19130280,46275803,40004096,40227609,1126128,19119244,19119179,45775632,40233295,42799224,36249547,40235382,46276238,40235349,40162637,19118856,40002628,40005761,40145223,1126879,40002820,42708313,42800258,19111349,40225821,1194283,40239597,42708016,40231873,1593996,19041284,45775691,19101551,40231883,40162563,40005762,43012474,40002857,43012472,44818347,19125683,19118857,40002691,40162495,19041576,1194754,43013346,19105599,19124767,40002632,40239644,40004085,40241541,40162596,1128795,19057277,40221902,19126262,19042800,43012465,19009291,40039900,40033572,42800343,40235368,40162584,40155710,19028090,42707993,19041612,40223233,1138029,40005702,40137516,19108729,40139752,19103757,45892547,40003486,40238715,40236694,40243252,40003521,19113396,40002834,1137230,42800362,19041285,44506411,40161008,40236684,40002852,40141088,19044374,42706840,19045934,40162471,19012057,1129233,1129232,19054933,19041322,1134508,19073366,19073408,40162529,19041467,44818348,46221788,19062009,19099179,40227607,40162552,40003522,40003512,19037341,40139755,1126539,40148362,40162589,40004088,40002769,19112957,40002709,40238738,19126263,19016435,40003530,46276240,1126827,40002756,40002804,40066383,43013305,19096600,19096432,42707333,40233298,35606615,19105368,40233306,44815752,42799147,40162540,40145222,40004045,19108636,40099996,40223191,40103369,40002821,44818383,43525711,40005721,19073402,19072158,40003562,40002651,19106627,46276039,19064664,43526549,40162576,42799070,40002770,19043332,19107265,19105748,19060557,40233292,40103224,42709000,19075754,40162603,964812,40003628,1135288,19037347,1113302,19005061,19042649,19124811,42902772,1103359,45775854,19084567,19123011,42799158,46275879,43013302,1135323,40221906,40138389,19042685,40233262,40221942,19113395,1128054,40010213,40231898,46275802,40142581,19092679,19010208,1126825,42799091,40238713,40162531,1127043,40238716,40004070,40240418,43011492,42707725,19041386,42705322,19018934,19118868,1129234,40162626,40145238,19132188,42708002,1127987,45775600,40145246,19034333,40162583,1127990,1193341,19086546,43013345,40233321,19015608,43013206,36249943,19102634,19041505,19105798,19028037,19042761,19105600,40005759,40071520,45775063,40002592,19107493,1134836,43526528,19054932,19076604,44816116,40233257,19041542,42901968,19092767,40003627,42800369,19024359,19105796,19106548,19028156,19041356,19005994,19037345,40162550,40162599,19010176,45775634,40002740,40231867,40002735,44816258,40221974,19123255,19062440,1127682,40229075,40162542,19118860,40221920,1127815,35606354,19105281,40005740,19060257,40162547,43012476,19119237,44816114,45775629,40005763,19107263,40005631,19126942,19002900,1126165,40005653,42800360,44815753,43012475,42708363,19113369,19028153,40002842,44785118,19118864,19075779,40162622,19126696,35604562,19024382,1125394,19135516,40162482,19110931,35604566,40237574,40103361,1126952,42706711,40227635,19118896,19073409,40233293,40240412,40005717,19024360,1128630,40005716,19061946,40241589,40002774,40003502,19058754,40004114,19129554,19096338,42800370,1126958,19082412,43532688,19063318,40233317,40233300,40162604,1153670,40003679,1165667,40162470,19105666,1193392,19118901,42707625,40162561,965070,40235362,19073694,19062932,1126874,19115388,19042683,1128482,19041654,40236703,19041288,19119724,40169411,1128871,40002847,40135208,40233296,43525684,40003531,43012489,1128060,19105797,40237594,1592181,44816142,42707943,46233708,43013271,42799196,43560442,19028713,40236692,19127656,40002796,19103474,43013292,19118936,19072315,40238694,40243235,40162530,1127498,702952,40227611,40148364,19107262,19019813,43012473,19126652,40002871,1126824,40003543,1135445,19118844,19042362,46275480,44785557,1127433,19015605,40005654,1113577,40150393,40017327,1135446,40231904,40004047,1128052,40002689,19005066,42706629,40237575,1127643,40169412,40103275,19107243,40145237,40128198,19135204,43532678,44816071,40136243,40240406,35606606,42901531,40103239,19071216,40003703,40002690,19041509,40238696,40229094,40162572,19033759,40221904,35604563,19041287,40128142,42708003,40233253,42707448,19118881,40065711,40131018,40240417,19108653,42708161,19024148,40003491,43013277,44818004,1146886,42873689,19124721,19041463,1129026,40227637,40130198,19105386,19016078,40099994,19105985,19041259,1127786,40004006,19046288,19001049,40145244,40221115,19041575,40003718,42901489,40144437,42901498,19118894,40103237,44815931,1137231,43013467,40231912,19133447,40162514,19103359,40005706,40138391,40233301,40003681,19072799,40003496,19092040,40004061,19106415,1126957,19124773,40221945,19107575,19079628,44815930,40004101,40004025,19107241,42799119,19092087,19121387,40003555,40004005,40002759,42707605,965066,40235395,40236715,1593754,19133267,19037348,42799092,40004004,1164226,1164228,19082316,40002737,40022973,19107232,19118874,40005705,40162551,40223198,1127687,40005773,40243250,1136757,45892865,43526693,46276242,1137526,40236678,40162566,40236702,40169440,40162614,42708385,40231869,19124716,19107480,19041177,19118923,40162600,19041578,40002869,40004104,44784663,40160333,40236681,19006734,43532516,40223200,1127688,40162492,40231900,40005695,40238723,40223223,19086541,40229110,40124793,40145247,43559973,40009896,19062700,19001042,40223205,1594002,19005991,40221917,19107244,40233276,40241581,40135187,1126959,19113352,40005703,42799116,40003609,19041252,42708030,40221913,40227591,45892864,40002848,44814617,40233305,40005711,1129189,40023624,1127644,19105462,19042171,40233250,40005798,19057847,19118931,40002656,46275883,40133147,1593999,40003501,40004059,19107521,40229122,40162502,42707446,40231856,40103227,40239637,40004094,40005642,19111755,19118528,19119726,42708159,45892330,40071521,44818320,42799136,40005710,40003498,40162509,40103373,1164270,45775024,42901150,40009857,19063234,40235381,40227642,40004038,19111632,42799188,40240419,40005750,19108613,40142606,19053797,40002701,43013293,19126928,19041651,19042826,1126953,19059057,19041176,19071215,40002861,40005715,19096692,19103361,40221969,40229068,19041258,19041501,19119720,19041611,40029882,1194284,40166149,19041354,19118922,43012467,40162621,1127042,19135521,42708128,45775856,40135538,40221034,40002623,40002624,19118852,44817918,40236677,44818376,42707624,40221929,40134128,40235292,19023394,40162539,19062578,40221919,40002761,40134212,19106282,40135182,40238724,19010144,42902311,40004007,19096195,19111628,43531897,42707628,42799074,40002762,1126566,1126563,19003127,40002758,43013296,19082081,40005709,1172207,40003500,19044433,40241585,44785558,19108618,42707846,19135545,19107659,19005970,40002739,43013344,45775630,43013318,40239569,19114931,40002863,40231905,40002843,42800412,40244101,43531899,40156496,43532952,42901718,19134676,40009894,19005993,40223214,1593667,40235314,40223202,19005995,40223201,19096574,19102640,40223209,40005650,40135541,43526525,40005765,1137229,40002677,19042765,19073491,19028573,42902258,35606613,19064133,19127495,40065712,19103709,19082416,40231924,36249945,40231714,19041257,40103241,19121424,1127528,40162582,40162587,42707813,40168529,42708869,19072201,40003574,19044372,43012466,19096147,19041891,19044377,1126499,19118858,40231907,1100166,40005753,40162565,40002630,1129102,40231866,42901868,40003508,40131031,19132079,40237565,40005797,40002836,40238739,42708773,19099865,40002669,1127578,40004092,40229105,19119727,19104216,40003487,1593998,44784664,42707806,46234325,40231880,19123142,35604560,43013285,19096148,1193313,19063541,1126902,19096974,40084339,19106027,19118850,19119220,40003695,19033710,1718874,19013117,19085706,40103365,40005782,1128698,43013294,40005675,40223254,40003550,40229134,19126655,40002786,40231899,19054934,40236714,40229095,19118934,43012493,19076475,19111352,40175515,1127493,40221966,40096468,19061859,40237568,40223247,40005651,40162535,19105982,40005785,40233318,40231895,19063317,19112785,19108533,35606618,19122357,40223211,44506410,40003490,19111635,40003509,19118933,40003541,42705787,42799105,40221952,19034973,40241594,1164268,40238710,19043335,19123406,40002841,1137991,40238736,40237582,1129104,40002849,19135525,36249944,40004062,40237576,42709262,19042766,19041577,40003563,42902256,19111759,40162568,40071519,43531895,19062724,40002789,19044434,43012456,1127526,19041282,19117723,19115993,19135552,40005662,40162553,40004054,19079968,19045236,40002694,1128059,40005735,19044378,19103554,1592585,40002844,1126564,40162577,40005649,40151320,1718528,965849,19041462,19001050,40243231,704974,19110003,40132296,1126537,40221940,40229062,40002626,19092053,46287284,46275459,40002837,1174919,1166235,40239584,42707919,40003698,19086572,40151272,40162522,40229131,44816065,43012449,19052540,40004048,40002616,40003535,19129452,40235366,40173510,40235318,40235367,45892546,19108431,40002675,1193233,40235321,19041384,40004039,40238709,19086542,40004084,43559956,40005708,40009892,40241575,43526548,19037349,40235380,1164151,43531860,40162496,40221968,40163815,19084048,40221914,19118940,19121390,19116433,46276041,1125395,19007723,40003646,19088960,40238735,40231901,42902460,1127575,44818283,19118928,19118929,19086574,19042798,40056910,43013311,19118908,40227589,40238719,40131016,40002860,19107495,40003484,40003536,1127611,40237587,1127523,40004081,43013219,40237600,40004075,42708306,44818334,19064791,42873690,40003565,19135692,40004003,19027738,19042823,19096674,40005792,45775851,40002763,42901795,19107517,40003510,1127107,19041353,40236705,43012468,40003549,19080853,1126538,40158695,19091501,19037382,40240409,40004055,1127019,1194159,19028858,42708163,19007724,40005731,40235384,19124624,40136244,19119722,1124004,40235374,40243242,19102641,40005698,19119719,19084222,40128271,40231915,19080854,40002730,43013320,43012130,42707668,42705324,40005719,40005658,40145225,40243247,40103372,42708006,40236676,40235339,19041283,19135110,40240413,40162560,40155942,1126494,40128140,19129487,40236688,40162518,40231920,19104607,40137518,19118943,40003649,42873512,40221013,40005751,40241542,19119221,19042821,19060558,40233315,40004065,19041355,40238726,40233314,1129106,19107146,40017332,19122789,40231921,1127684,45892549,44818050,42898298,19105405,44817984,40233289,44506605,19083358,19118865,1113267,19121095,40221938,40223238,40237579,40003674,19028035,42708156,1123909,40003693,19119254,44818014,19102633,40243245,19042791,40128272,43525455,40238730,40003729,19006000,40221931,40162628,42705166,40004058,19133292,40231882,40130203,40162524,40221950,42707827,40133145,1125360,19041653,19119241,42707626,40058007,40236690,40241587,1126497,40003537,19119178,40235372,19118942,42799098,40137949,1164302,19063402,43526546,19007799,43532679,40005805,40155160,40221918,40233307,44818337,1135359,42902342,40235309,40002874,40005768,40162607,40237586,40005678,40153523,40237601,19075780,45775506,19106831,1174918,40011362,40012628,40162546,40003683,40227627,19046903,19108614,40078623,1127047,40241568,19064141,40235361,40096470,1125359,19069104,40135194,46275882,19118882,42902251,1126909,19082414,40005760,40003610,19072435,46276040,40238704,19125986,40162475,42705321,40236713,40162595,19118925,19054985,40002620,43560440,965846,40002667,40004072,40002811,19063075,19073812,46275804,1127574,19029322,19126394,19099651,19105369,19128899,40223225,19041424,40004023,40220874,40235320,40244099,19108620,19069543,40231887,40003488,44818289,35602878,19072175,40162544,43525949,19119216,40002719,19063074,42707868,19118903,40005766,40162609,19102075,1127577,19118891,40231945,40005736,40233297,19118907,40235388,40223232,40003542,42799067,40103229,40236707,40162592,42902185,40223195,40002802,19057058,40004033,40237558,40162567,40231906,19075729,42708129,40005776,40162580,19118932,1127529,40244098,40223251,19072174,42799087,40233280,19107284,40162549,40236675,1126747,40235307,40229082,19127003,40238712,40003570,40128196,40221120,19105497,40142587,43012115,40237549,19123141,40237552,40002793,42901966,19130281,40223226,965064,1195423,19107592,40142589,19065577,42901609,19041199,19129413,19126691,40244097,40239596,40237551,40017333,40003539,19106393,40002801,1128830,19073364,1126461,40003581,19012741,1128222,40003534,40162521,40071522,19123261,19046076,40235329,42709031,44814263,40005685,40235375,40145236,40231930,19028574,19118853,19109781,19119247,40039717,19105466,44818335,965509,19105795,40162588,40229086,964407,40012311,1128057,40223194,19059092,45892866,19118863,40005718,19104128,40221908,40221951,19005257,40002703,40005726,40229081,19024381,19053045,40229120,40221939,40236682,40004027,40002767,19044373,40002670,19042615,44785119,40223243,43013334,19110004,19118888,40135537,40231928,43526527,19117119,19028905,40004028,43526526,1593757,40162585,42801313,43531898,40004100,19072235,43013297,1718090,19109336,43559966,40231855,45775807,40162570,19072437,19041461,40229093,42873369,1592594,19060939,19125299,40235301,40221117,19041615,19009292,19080855,40002781,40237571,40231870,43013333,19104575,19032372,40168532,1128349,40162606,19107790,42707529,40223252,1128700,1127500,36249545,35605901,1128730,40233279,40223228,40236708,19062772,19073492,40002692,40004068,40163814,40005794,40162497,19087814,19064698,1194282,46275881,40003505,19119222,40004035,40003485,19103362,44818333,19122380,42799090,19067297,19119250,40059916,1128460,1719035,40002866,40004053,19085989,19029171,19062185,40162579,40221905,40236699,40005652,40003516,19061979,19041286,19042500,1126166,40002642,19041428,40231932,40004018,19106026,40002639,19058755,1128524,1126903,40154296,19112602,40238725,40227614,19047710,40128150,40162573,19123905,40233322,19072206,40235310,19041464,19107520,19073472,42708149,42799225,19118944,40103272,35604565,1113563,19107573,44785555,1128056,19105337,40004095,40002851,40002728,40005791,40131017,43012495,35606324,40002717,42902223,40162501,45775508,40005733,19107091,40148361,44814262,40004108,40235353,40003697,40237605,19062264,42707332,19029323,19016439,19107808,19008387,40003492,19119255,19136015,40221979,40002744,36249947,1127650,42706839,40158765,40237585,45775601,40128158,1718089,40002749,19132631,45775852,19041383,19086607,19107374,44818359,19006295,19102533,19118895,40231857,43012079,40166844,19041541,40005775,40237569,40004010,19107409,40002795,40162527,19119235,40004024,1127954,40221033,40223248,40003548,40003619,19105749,19065255,19107558,42873273,19128788,19110959,40221960,45775062,40002686,42705386,19118899,19028904,40162516,19108433,43532887,40171531,749965,45775023,40145245,19062967,43012459,40225818,19108429,40002683,42873688,40231916,43532898,40231875,19125987,19062752,40004021,19118869,40236696,43013314,1137460,19120990,19119019,19041579,40231919,40128143,40235385,40221907,1127689,45774840,19045936,40128171,40221941,44818313,1146891,40235315,40227628,40153549,19041580,40239645,1128728,19087815,42799076,40002731,19126690,40003626,42707331,40238705,46275338,19125518,19062100,19054931,19105406,19109337,19114928,40003654,42708000,40242522,19110002,40235319,40003524,19076426,40237583,42709293,40169427,19119236,40005732,43013469,42707779,19116801,19028854,42799143,40239634,19122785,40002590,43526544,40236686,45777021,40169425,19034977,19041260,42705696,40005677,19021146,40134210,42707714,19042822,965065,19108623,45776384,19028125,40238718,1115065,19059196,42800331,42898302,40231871,45776410,19073268,40002722,19092702,19056115,40002604,40002627,1164143,19117379,42707819,40244110,40221978,40034240,42706712,40003556,19011669,19028821,40002747,42901546,19118867,19006021,40005679,40233274,40099991,40002714,40227605,40124834,40003564,40235352,42901488,40236672,45892404,40004011,19072159,40162519,40221948,19111631,19108644,40004008,40162484,40239567,40233320,43560446,19124813,43560417,19135126,19105492,44817980,19041502,19125547,1168152,40221959,44815929,1129103,19118879,19119018,40237562,19042792,44818054,46275805,40241605,40004026,19028642,40002679,42705629,40138392,40221963,1164334,42705695,19123143,19122636,19132556,1128197,19042364,40004082,35604561,43013335,40235383,40002862,43559974,40004066,42707817,19127533,19116549,1128870,19130282,44817998,40241583,1100168,40236683,19107604,42902221,40175528,45776276,19041359,19009293,19024383,40004052,40235360,40103278,40005784,19075730,19122379,19107630,44816072,1103360,40039716,40162520,40003493,19118846,1195334,1126492,19062931,40235394,19108928,40002799,42708782,19075753,40003650,40002643,19119252,19123377,40071518,40229085,40002734,40238697,43560448,19118862,40002748,40002673,40005645,40241595,43011827,1592489,19072176,19085696,40005682,1193312,19093038,43012455,40150372,40231859,35604547,42901497,35606327,19115210,40233290,19107268,19118849,40162532,19087510,40229145,19092178,40238660,40162575,40005799,40139760,40133146,42799080,40135183,19105504,19120779,40003511,40002657,40235370,1164225,40003687,40235336,19119257,1592589,1126569,40223230,40223241,40004014,40223206,42800335,19033732,40235390,40002619,40005669,1126744,19118871,40005746,19075777,19119238,40162620,45775741,43559970,19118872,19029147,40004103,40162468,19067292,19028316,42708997,40003686,1593751,44816262,42800371,19113368,40233258,19107519,40003518,42799588,40235341,40162505,19102449,40221957,1127499,35604546,45775507,40227598,19105562,19041281,40134079,40162605,40003597,19044431,19093394,19041321,40004118,40002594,40162507,40004046,40004107,40005767,40005800,19122503,1127610,40029881,1164269,19107264,1100169,1127496,40227613,44816134,19080534,1129031,40231872,1126982,19073381,40241564,19119715,19134836,40002792,40004015,19041426,1127434,40227593,40103279,35604558,1127018,19103402,40011644,40231886,40235365,1127016,1126620,40002625,1155308,40235350,1128225,1718869,40003608,19119848,40242530,43013278,40003573,40235378,19028036,19123256,40221114,19079627,19118937,42898299,40223253,19042522,40004057,19042767,40167824,40168485,40241544,40003622,19041256,19041352,40162612,40231938,19074854,44817982,1128016,40237550,40162474,1127791,40005801,40099716,19072178,19132187,43013202,40124795,40235379,19118861,43526541,40240416,19044432,40140509,19044376,45775599,965238,19119258,42799593,40221946,19105496,19105491,40223240,19041255,19041389,19058752,40236711,19105494,40142592,42873699,19041657,19041357,40004120,40144438,1127525,43013279,40005723,40131029,19118851,19121386,1127893,44785120,40003689,40233299,40003540,19098240,40242532,42898296,42799644,44818352,19096480,40058008,1134841,40162616,40004112,43559971,19095054,40005724,40237604,19107494,40235293,40244100,40162558,19073397,40162488,40003482,40236698,40005689,1126750,40162510,40235312,40223231,42800355,40004076,40221901,40162618,40162557,40231891,40137971,19105495,42901970,19080535,42707630,40236679,40002788,40162615,42898300,19075081,19108617,40002773,19106392,40005742,40096469,40002738,40233256,19028151,1126872,19106608,40163813,40229106,40241569,42799148,40043637,1195374,40238728,40005734,40223199,40141982,724425,19119717,40137952,19118906,40019011,1126565,19108227,1126460,19105531,1126904,19124774,35606353,40162508,40003705,40005684,19107145,42902249,40162581,19125359,19118883,40155105,19105667,40002678,40229103,40158761,40162601,1126984,19116000,19073369,44507435,40005725,44818051,40135195,19107215,19102661,40002693,42707918,43526539,40005793,1126536,19106683,19118848,19122977,40225819,40162491,40223227,40231865,40162632,40128151,19009691,19041573,19079925,19119245,40227590,43532897,19106908,40103371,19111633,19118909,19041253,1140751,40241582,19118893,19080551,40142614,1127813,40005745,40231917,19073438,42799075,19041323,40133154,40002711,40229096,19118939,19096975,40003641,44814284,40223212,40004032,40004067,40003617,44817909,40239620,19066373,19125444,19096978,19115999,19072232,1125320,42707824,1113567,43012078,42707629,19041330,40002617,19073363,40003684,40002685,42705630,40012268,1130613,40220876,35604559,40003696,1126826,40162611,40002833,19073529,42901495,1164231,43560441,40231943,35604567,40221922,19073559,40003725,19091785,19128787,19119246,40221943,19075310,45892329,42800256,40003586,19073365,43013295,19065640,40005737,1592586,40002875,1194495,40003720,40137950,40002850,40236693,1128664,19128413,19119082,40003717,40005752,40102695,40003525,40231931,19107591,45775061,40162476,40003569,45775501,19088707,1592330,40231935,40153555,40004110,19119248,40148360,1126498,43526540,40166846,1127986,40243233,42873591,40004080,19062040,42873698,19108625,42799153,19064666,42900562,1128663,40236700,40237566,19107522,44785881,19135528,19042797,40003538,40233313,19107269,19041387,40221909,40162630,1129144,1127530,35602868,40099998,40236706,40151315,19124723,19071301,40002732,19041358,19122358,40002855,19095057,40236701,40004086,40223250,19079924,19047295,44818336,1126535,40164003,19101890,40103367,43560445,19073553,40169379,45775693,40162469,1593668,40162534,19072160,40004013,19041503,35606614,40233249,40241578,40003723,1126495,45776386,19028348,40157251,45775692,40221955,19041470,19100296,19005281,40227612,42873195,40005783,40059884,19122978,1126878,40005779,40003572,40005674,40002807,19007800,19042799,40005670,1128696,19132105,42799077,40012271,40002787,40162619,40241588,1718532,43011493,19115996,40004063,40002702,19080552,19073591,42901537,40231923,43012492,19034423,40229073,40231909,1164335,40004109,40162545,40220835,19127002,42707435,19105387,40241586,19113353,1127899,40004064,1140384,40003483,40168528,42709264,42708992,40171435,42799150,40162526,43532883,40002809,40003680,40103375,40229102,40140510,19041465,40162594,19105561,40162617,40017331,1718529,19116253,19105750,40135534,1128828,40162466,1126911,19092059,19119180,40166174,19112845,19073270,43532676,40004044,44818345,40134228,1126616,42800346,19118945,40161009,40004089,40162574,19129488,40155168,40005749,40240415,44818382,19041655,40009853,40002853,1135358,19119242,40003497,19041251,19041425,19047696,40177555,40138387,19063636,1164230,40002727,19105577,40004012,42707902,43012469,1592199,42708991,40002816,19118887,19108638,40002688,19063151,1127522,40005744,40145224,1718871,42707328,19041326,1127686,40169410,44818381,46221786,40005687,40238714,19030205,19123271,43532581,19111636,40005676,40162590,40233285,40231926,44785559,19103363,42800359,19101635,1128801,1126877,19041289,19116562,42629533,40002705,19033731,19099864,19119847,40004105,44817912,40236689,19072204,43013319,40221970,19046220,44506378,19107267,42901806,42629298,40002621,40092357,40235371,40005680,40002603,1164224,40002859,1164144,19042521,1127783,19073433,42708042,19048871,40231939,40231861,19054753,35603505,1194110,19080717,19073500,40221972,19135549,19041421)

In [ ]:
vaccine=(529343,19028825,19133778,40129801,42800500,45775640,19132353,19130406,529490,46276200,19132352,46275156,46276292,46276279,19028826,46276202,45775645,46276422,19132299,19133451,528323,45775644,19102281,46275089,40148796,19067405,40156660,528353,40153982,45892099,46275092,40086962,40094349,42903195,19082708,46276277,45892098,19131878,46275088,42800063,19130408,40150267,529377,19104131,40090138,529374,19131876,19028827,46275238,42800266,40153467,42902378,529484,19112113,19131659,40156519,529303,529488,19136026,19111871,42800472,40156527,40152821,40153466,40149494,46275090,559532,19023455,529347,19133777,40243773,46275140,530007,19065535,19077988,19120571,19056508,590676,42902407,19120570,46276268,46275142,529376,19067185,19065534,46276418,46275153,19122004,40243771,523290,42800061,42800480,40173988,509103,19058439,19028828,42800484,46276281,19131358,40045653,45775643,45892101,529348,551949,19123085,19104170,40064425,19131940,42902512,529461,529485,19067186,40045078,19104287,523289,19130407,42800502,46275085,40081055,42902411,19104166,544411,594249,529455,46275086,40156522,40153533,46275241,19133426,40046017,42902525,40086961,19065533,46275091,529653,19130196,46276206,46275236,529652,19027665,19064695,529411,46276421,40055282,529657,42800485,42902481,46275237,19063339,40065975,590675,19132298,42708198,523288,523212,42800478,19032599,19133449,19097994,46275139,40156525,19102417,19122005,46275154,42903371,19023428,40173449,40157874,19093066,523202,46276423,46275094,40021647,532274,40155131,40034293,40150268,46275231,19102282,19129484,42800027,42800501,40243219,42800267,19129379,46275244,528324,586325,40153981,42800486,46276280,40148795,523204,42903243,529656,46276308,19104174,46275242,42800461,42903199,19067404,19130194,40148693,19099513,529483,529482,19034102,19131880,19132128,586324,19131938,45892100,40150353,46275141,46275093,46276207,40034290,42800264,19131356,19133774,19132300,46276295,46221248,46275143,1593421,19089797,1593422,19132297,19077990,46276201,19129406,40173953,19035048,19122006,46275232,40153532,19112111,42800265,19089501,19130195,42902325,46276203,40148731,42800476,46275243,19028830,19129482,40034292,529346,46276419,46275155,40150352,42903058,40034345,523287,40157875,19102280,19023427,42902564,42800271,46276285,42800477,19096253,40046018,529456,46276282,40148692,523215,19133452,40124278,19115028,596876,42800270,45775636,594250,46275229,40034291,40055281,45775646,523213,19133773,46276269,42903320,594272,509104,19100898,509106,42800062,40173989,40156658,19027664,45892095,19131459,19133775,529342,19097917,19131460,40001983,46276278,40055280,42902482,45775639,42800462,46275084,19040342,46275138,46275144,551974,42902398,40134143)

In [ ]:
aceinhibitor=(40172814,40163268,1334535,40163767,42800850,40171562,19117559,1342476,40099414,19115847,40165770,974201,40096447,19054306,40171547,40099416,40163274,1342473,19013173,19003828,19037085,19102107,40020853,40142680,1342503,19116428,19079436,19109176,42801012,40171559,19042717,40044819,1334497,40163277,19003086,40044841,40172815,19029027,40171561,1308250,19045583,19117103,19106535,19021201,40165778,19122327,40184277,19047476,40098152,40172813,42801015,1341927,19082321,40172078,1341968,1308309,19107373,974447,19120513,19050216,19038727,19117562,1308305,40171678,40165783,974852,40171679,40171668,1308216,19102170,1331241,19047474,40172822,40171552,40172819,19003854,974854,40068358,40171558,1331265,40171553,19040059,40000499,40098149,40171556,19003126,40171555,40163756,19044704,19054305,1334495,40165773,40167240,40171661,1342001,40154151,19022243,1342506,19036750,40163752,45892984,40165762,19120514,40171664,19050316,19078104,40171418,19101751,40171663,19040058,40101444,19025174,42801013,40171675,19102169,1363749,19025234,40163754,19025180,19106538,19115849,19074673,19087486,40020854,19103511,19036771,1328713,1334492,19117057,19017082,19117555,19025231,19025175,1334461,19050315,19036749,40020858,19042713,19134740,40165750,19038728,40171667,42800851,40171564,19102051,40127543,1334536,40165790,40163270,19112468,19033405,40000501,1341998,19021168,19113587,40068359,40163768,19115736,19017026,19038726,19116429,40163273,40171659,40165765,19117560,19117102,43560163,40155308,40165754,1331266,40184216,19132832,40165751,1342507,19116427,19040051,19100828,19117554,40163764,19120510,19111733,40163753,40027598,40171551,40184215,1331264,40171420,40034890,45892986,40171419,40099445,19109177,40221240,40107388,40056578,40172820,19103788,1342474,40042338,1331235,19017285,40100453,40083796,42801010,40171560,40165775,19044705,40068357,40096446,19125669,45892981,40039229,1342026,40221246,19127397,1334496,40163751,1342023,19003852,40171417,19113826,1328742,40165771,19009182,19033404,40027615,19078353,40042334,40165779,42801014,40020855,40098151,19033409,40163758,19081025,40184213,19106536,40165768,40165759,19057857,974200,19050318,40163760,42800843,19057855,19033403,19017284,45892993,19062727,19078351,19050311,42800845,40172818,19101749,45892987,40171672,40165760,40020852,40020850,45892983,40160142,1342472,40027614,40171658,1340159,19057853,19088842,19003856,40034888,40042350,19132831,40020857,40138364,45892980,45892989,19102053,40171662,974171,40039209,19003855,19040057,40165774,40171670,19044706,19117056,40044817,19037084,19040056,19065449,19117068,40165782,40184214,45892990,40172821,19040053,40171676,19106537,19106542,40163750,19120509,19113588,40171673,40163766,974443,40171683,19120511,19009181,40165752,19096771,1308251,19129686,1341969,40172816,19025178,40163755,19047475,19106539,40163269,19115730,40163267,40171416,40221245,19117556,40163757,19013175,19132833,974192,19115850,40042000,19047477,19078137,19025232,1353820,40154690,19127399,19009160,1342475,19021170,19045770,19117055,1340160,40039228,19106534,1340161,40034889,40163271,19113825,19101748,40221242,1373225,40184154,40165791,40020851,19085108,40000502,978575,1308304,19104348,40168022,19037083,19074671,40020558,19045846,40163761,40171415,40143577,974199,19025177,19101750,40165785,40168021,40171671,19047473,19061252,19013172,40165781,19003853,43560166,19109178,1340153,19022241,19033410,43560162,40171680,40083797,974853,40096153,19040055,19021169,42801016,40172824,1342505,40083794,40171563,19025179,40171548,40165777,40057033,40172825,40165757,1342027,40171674,19080218,40169643,19067557,40099444,19080128,40221249,19040052,19129687,19074672,45892992,40171413,45892979,40042335,19003857,19033406,19017025,40165763,19025235,19120321,40163272,1308306,43560165,1342441,40041997,40163749,40171554,40171549,19003830,40099446,40165780,40163762,40045124,19100830,19127396,19102052,19054309,1340182,19003851,40020555,40165787,42800852,42800847,19054304,40163276,19036773,1334460,1340157,40037281,40042337,1334456,19017027,19117101,42800842,19017081,40165767,40045122,40171550,40163266,19087634,19040054,1342504,40115015,40171557,974170,40171669,40171660,19003829,19072530,42801011,43560167,40171681,40165753,19117067,42800844,43560164,19117561,19083078,40042339,19025176,42800846,1331240,40057031,19078105,1340128,19017283,40171666,19115846,40049453,19054308,40167616,40171684,40172817,40172823,40167615,1334494,19080129,19022519,40184278,19072529,40165786,1308307,19117066,19050220,19100841,40165772,19100829,19106543,1342439,19120512,19133002,40101545,40008942,19112467,40165748,40163748,40163275,45892991,1331267,40000500,40098150,1334498,42801017,40163142,1308249,19121746,19050314,40165764,1308308,40171677,1335471,40044818,40171657,40221243,45892985,40020856,1334459,19078352,19072551,40171682,19082957,19022242,19054307,19107180,45892988,19132834,19025233,19106830,40171665,19017083,1308221,40163765,40165789,19125668,40165776,40165769,40171412,19127398,19013174,40163759,974169,1310756,40165755,19129688,40163763)

In [ ]:
drugs=antibiotics+ccb+diuretics+opioids+statins+msknaids+oralhypo+painnsaids+vaccine+aceinhibitor



## Antibiotics

In [ ]:
antibiotics_standard_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_total_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            WHERE drug_concept_id in {}
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_well_defined_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.route_concept_id
            WHERE 
                 t3.standard_concept="S"
                 and
                 drug_concept_id in {}
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        route_well_defined_row,
        route_total_row,
        round(100*(route_well_defined_row/route_total_row),1) as route_success_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,antibiotics,DATASET,DATASET,DATASET,antibiotics,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
antibiotics_standard_df.shape

In [ ]:
antibiotics_standard_df

In [ ]:
# antibiotics_standard_df.to_csv("data\\antibiotics_standard_df.csv")

## Ccb

In [ ]:
ccb_standard_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_total_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            WHERE drug_concept_id in {}
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_well_defined_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.route_concept_id
            WHERE 
                 t3.standard_concept="S"
                 and
                 drug_concept_id in {}
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        route_well_defined_row,
        route_total_row,
        round(100*(route_well_defined_row/route_total_row),1) as route_success_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,ccb,DATASET,DATASET,DATASET,ccb,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
ccb_standard_df.shape

In [ ]:
ccb_standard_df

In [ ]:
# cbc_standard_df.to_csv("data\\cbc_standard_df.csv")

## Diuretics

In [ ]:
diuretics_standard_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_total_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            WHERE drug_concept_id in {}
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_well_defined_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.route_concept_id
            WHERE 
                 t3.standard_concept="S"
                 and
                 drug_concept_id in {}
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        route_well_defined_row,
        route_total_row,
        round(100*(route_well_defined_row/route_total_row),1) as route_success_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,diuretics,DATASET,DATASET,DATASET,diuretics,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
diuretics_standard_df.shape

In [ ]:
diuretics_standard_df

In [ ]:
# diuretics_standard_df.to_csv("data\\diuretics_standard_df.csv")

## Opioids

In [ ]:
opioids_standard_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_total_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            WHERE drug_concept_id in {}
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_well_defined_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.route_concept_id
            WHERE 
                 t3.standard_concept="S"
                 and
                 drug_concept_id in {}
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        route_well_defined_row,
        route_total_row,
        round(100*(route_well_defined_row/route_total_row),1) as route_success_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,opioids,DATASET,DATASET,DATASET,opioids,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
opioids_standard_df.shape

In [ ]:
opioids_standard_df

In [ ]:
# opioids_standard_df.to_csv("data\\opioids_standard_df.csv")

## Statins

In [ ]:
statins_standard_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_total_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            WHERE drug_concept_id in {}
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_well_defined_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.route_concept_id
            WHERE 
                 t3.standard_concept="S"
                 and
                 drug_concept_id in {}
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        route_well_defined_row,
        route_total_row,
        round(100*(route_well_defined_row/route_total_row),1) as route_success_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,statins,DATASET,DATASET,DATASET,statins,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
statins_standard_df.shape

In [ ]:
statins_standard_df

In [ ]:
# statins_standard_df.to_csv("data\\statins_standard_df.csv")

## Msknaids

In [ ]:
msknaids_standard_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_total_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            WHERE drug_concept_id in {}
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_well_defined_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.route_concept_id
            WHERE 
                 t3.standard_concept="S"
                 and
                 drug_concept_id in {}
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        route_well_defined_row,
        route_total_row,
        round(100*(route_well_defined_row/route_total_row),1) as route_success_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,msknaids,DATASET,DATASET,DATASET,msknaids,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
msknaids_standard_df.shape

In [ ]:
msknaids_standard_df

In [ ]:
# msknaids_standard_df.to_csv("data\\msknaids_standard_df.csv")

## oralhypo

In [ ]:
oralhypo_standard_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_total_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            WHERE drug_concept_id in {}
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_well_defined_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.route_concept_id
            WHERE 
                 t3.standard_concept="S"
                 and
                 drug_concept_id in {}
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        route_well_defined_row,
        route_total_row,
        round(100*(route_well_defined_row/route_total_row),1) as route_success_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,oralhypo,DATASET,DATASET,DATASET,oralhypo,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
oralhypo_standard_df.shape

In [ ]:
oralhypo_standard_df

In [ ]:
# oralhypo_standard_df.to_csv("data\\oralhypo_standard_df.csv")

## painnsaids

In [ ]:
painnsaids_standard_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_total_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            WHERE drug_concept_id in {}
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_well_defined_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.route_concept_id
            WHERE 
                 t3.standard_concept="S"
                 and
                 drug_concept_id in {}
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        route_well_defined_row,
        route_total_row,
        round(100*(route_well_defined_row/route_total_row),1) as route_success_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,painnsaids,DATASET,DATASET,DATASET,painnsaids,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
painnsaids_standard_df.shape

In [ ]:
painnsaids_standard_df

In [ ]:
# painnsaids_standard_df.to_csv("data\\painnsaids_standard_df.csv")

## vaccine

In [ ]:
vaccine_standard_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_total_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            WHERE drug_concept_id in {}
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_well_defined_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.route_concept_id
            WHERE 
                 t3.standard_concept="S"
                 and
                 drug_concept_id in {}
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        route_well_defined_row,
        route_total_row,
        round(100*(route_well_defined_row/route_total_row),1) as route_success_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,vaccine,DATASET,DATASET,DATASET,vaccine,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
vaccine_standard_df.shape

In [ ]:
vaccine_standard_df

In [ ]:
# vaccine_standard_df.to_csv("data\\vaccine_standard_df.csv")

## aceinhibitor

In [ ]:
aceinhibitor_standard_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_total_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            WHERE drug_concept_id in {}
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_well_defined_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.route_concept_id
            WHERE 
                 t3.standard_concept="S"
                 and
                 drug_concept_id in {}
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        route_well_defined_row,
        route_total_row,
        round(100*(route_well_defined_row/route_total_row),1) as route_success_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,aceinhibitor,DATASET,DATASET,DATASET,aceinhibitor,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
aceinhibitor_standard_df.shape

In [ ]:
aceinhibitor_standard_df

In [ ]:
# aceinhibitor_standard_df.to_csv("data\\aceinhibitor_standard_df.csv")

## Drugs

In [ ]:
drugs_standard_df = pd.io.gbq.read_gbq('''
    WITH
        data1 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_total_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            WHERE drug_concept_id in {}
            GROUP BY
                1
        ),

        data2 AS (
            SELECT
                src_hpo_id,
                COUNT(*) AS route_well_defined_row
            FROM
               `{}.unioned_ehr_drug_exposure` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_drug_exposure`)  AS t2
            ON
                t1.drug_exposure_id=t2.drug_exposure_id
            INNER JOIN
                `{}.concept` as t3
            ON
                t3.concept_id = t1.route_concept_id
            WHERE 
                 t3.standard_concept="S"
                 and
                 drug_concept_id in {}
            GROUP BY
                1
        )

    SELECT
        data1.src_hpo_id,
        route_well_defined_row,
        route_total_row,
        round(100*(route_well_defined_row/route_total_row),1) as route_success_rate
    FROM
        data1
    LEFT OUTER JOIN
        data2
    ON
        data1.src_hpo_id=data2.src_hpo_id
    ORDER BY
        1 DESC
    '''.format(DATASET, DATASET,drugs,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
drugs_standard_df.shape

In [ ]:
drugs_standard_df

In [ ]:
# drugs_standard_df.to_csv("data\\drugs_standard_df.csv")

In [ ]:
antibiotics_standard_df=antibiotics_standard_df.fillna(0)
ccb_standard_df=ccb_standard_df.fillna(0)
diuretics_standard_df=diuretics_standard_df.fillna(0)
opioids_standard_df=opioids_standard_df.fillna(0)
statins_standard_df=statins_standard_df.fillna(0)
msknaids_standard_df=msknaids_standard_df.fillna(0)
oralhypo_standard_df=oralhypo_standard_df.fillna(0)
painnsaids_standard_df=painnsaids_standard_df.fillna(0)
vaccine_standard_df=vaccine_standard_df.fillna(0)
aceinhibitor_standard_df=aceinhibitor_standard_df.fillna(0)
drugs_standard_df=drugs_standard_df.fillna(0)

In [ ]:
antibiotics_standard_df=antibiotics_standard_df[["src_hpo_id","route_success_rate"]]
ccb_standard_df=ccb_standard_df[["src_hpo_id","route_success_rate"]]
diuretics_standard_df=diuretics_standard_df[["src_hpo_id","route_success_rate"]]
opioids_standard_df=opioids_standard_df[["src_hpo_id","route_success_rate"]]
statins_standard_df=statins_standard_df[["src_hpo_id","route_success_rate"]]
msknaids_standard_df=msknaids_standard_df[["src_hpo_id","route_success_rate"]]
oralhypo_standard_df=oralhypo_standard_df[["src_hpo_id","route_success_rate"]]
painnsaids_standard_df=painnsaids_standard_df[["src_hpo_id","route_success_rate"]]
vaccine_standard_df=vaccine_standard_df[["src_hpo_id","route_success_rate"]]
aceinhibitor_standard_df=aceinhibitor_standard_df[["src_hpo_id","route_success_rate"]]
drugs_standard_df=drugs_standard_df[["src_hpo_id","route_success_rate"]]

In [ ]:
antibiotics_standard_df=antibiotics_standard_df.rename(columns={"route_success_rate": "antibiotics_success_rate"})
ccb_standard_df=ccb_standard_df.rename(columns={"route_success_rate": "ccb_success_rate"})
diuretics_standard_df=diuretics_standard_df.rename(columns={"route_success_rate": "diuretics_success_rate"})
opioids_standard_df=opioids_standard_df.rename(columns={"route_success_rate": "opioids_success_rate"})
statins_standard_df=statins_standard_df.rename(columns={"route_success_rate": "statins_success_rate"})
msknaids_standard_df=msknaids_standard_df.rename(columns={"route_success_rate": "msknaids_success_rate"})
oralhypo_standard_df=oralhypo_standard_df.rename(columns={"route_success_rate": "oralhypo_success_rate"})
painnsaids_standard_df=painnsaids_standard_df.rename(columns={"route_success_rate": "painnsaids_success_rate"})
vaccine_standard_df=vaccine_standard_df.rename(columns={"route_success_rate": "vaccine_success_rate"})
aceinhibitor_standard_df=aceinhibitor_standard_df.rename(columns={"route_success_rate": "aceinhibitor_success_rate"})
drugs_standard_df=drugs_standard_df.rename(columns={"route_success_rate": "drugs_overall_success_rate"})
liste=[antibiotics_standard_df,ccb_standard_df,diuretics_standard_df,opioids_standard_df,statins_standard_df,msknaids_standard_df,oralhypo_standard_df,painnsaids_standard_df
       ,vaccine_standard_df,aceinhibitor_standard_df]

In [ ]:
for i in liste:
    drugs_standard_df=pd.merge(drugs_standard_df,i,how="outer",on="src_hpo_id")
drugs_standard_df

In [ ]:
drugs_standard_df.to_csv("data\\q5_Integration_Routes_Select_Drugs.csv")

#  Integration of Drug Concept Sets:

## antibiotics ingredient

In [ ]:
# antibiotics_ingredient = pd.io.gbq.read_gbq('''
#     SELECT
#         DISTINCT c.concept_id AS ingredient_concept_id, c.concept_name AS ingredient,COUNT(*) AS count
#     FROM
#        `{}.unioned_ehr_drug_exposure` AS de
#     JOIN
#       `{}.concept_ancestor` ca
#     ON
#       de.drug_concept_id = ca.descendant_concept_id
#     JOIN
#       `{}.concept` c
#     ON
#       ca.ancestor_concept_id = c.concept_id
#     WHERE
#       c.concept_class_id = 'Ingredient'
#     AND
#         de.drug_concept_id IN {}
#     GROUP BY 1,2
#     ORDER BY count DESC
#     '''.format(DATASET, DATASET,DATASET,antibiotics,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
#                                     dialect='standard'
# )
# antibiotics_ingredient.shape

In [ ]:
# antibiotics_ingredient

In [ ]:
# top_antibiotics_ingredient=tuple(antibiotics_ingredient["ingredient_concept_id"].head(20))

In [ ]:
# top_option_size=min(len(top_antibiotics_ingredient),20)

## antibiotics ingredient by sites

In [ ]:
# antibiotics_ingredient_site = pd.io.gbq.read_gbq('''
#     SELECT
#         mde.src_hpo_id,
#         COUNT(DISTINCT c.concept_id) AS count
#     FROM
#        `{}.unioned_ehr_drug_exposure` AS de
#     JOIN
#       `{}.concept_ancestor` ca
#     ON
#       de.drug_concept_id = ca.descendant_concept_id
#     JOIN
#       `{}.concept` c
#     ON
#       ca.ancestor_concept_id = c.concept_id
#     JOIN
#       `{}._mapping_drug_exposure`  AS mde
#     ON
#         de.drug_exposure_id=mde.drug_exposure_id
#     WHERE
#        c.concept_id IN {}
#     AND
#         de.drug_concept_id IN {}
#     GROUP BY 1
#     ORDER BY count DESC
#     '''.format(DATASET, DATASET,DATASET,DATASET,top_antibiotics_ingredient,antibiotics,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
#                                     dialect='standard'
# )
# antibiotics_ingredient_site.shape

In [ ]:
# antibiotics_ingredient_site

In [ ]:
# antibiotics_ingredient_site['success_rate']=round(100*antibiotics_ingredient_site['count']/top_option_size,1)
# antibiotics_ingredient_site

In [ ]:
# antibiotics_ingredient_site=antibiotics_ingredient_site.rename(columns={"success_rate":"antibiotics"})
# antibiotics_ingredient_site=antibiotics_ingredient_site[["src_hpo_id","antibiotics"]]
# antibiotics_ingredient_site

## ccb ingredient

In [ ]:
# cbc_ingredient = pd.io.gbq.read_gbq('''
#     SELECT
#         DISTINCT c.concept_id AS ingredient_concept_id, c.concept_name AS ingredient,COUNT(*) AS count
#     FROM
#        `{}.unioned_ehr_drug_exposure` AS de
#     JOIN
#       `{}.concept_ancestor` ca
#     ON
#       de.drug_concept_id = ca.descendant_concept_id
#     JOIN
#       `{}.concept` c
#     ON
#       ca.ancestor_concept_id = c.concept_id
#     WHERE
#       c.concept_class_id = 'Ingredient'
#     AND
#         de.drug_concept_id IN {}
#     GROUP BY 1,2
#     ORDER BY count DESC
#     '''.format(DATASET, DATASET,DATASET,cbc,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
#                                     dialect='standard'
# )
# cbc_ingredient.shape

In [ ]:
# cbc_ingredient

In [ ]:
#  top_cbc_ingredient=tuple(cbc_ingredient["ingredient_concept_id"].head(20))

In [ ]:
top_ccb_ingredient=(1332418,1318853,1307863,1353776,1318137,1328165,1319880,1326012,1319880,1319133,19015802)

In [ ]:
top_option_size=len(top_ccb_ingredient)

## ccb ingredient by sites

In [ ]:
ccb_ingredient_site = pd.io.gbq.read_gbq('''
    SELECT
        mde.src_hpo_id,
        COUNT(DISTINCT c.concept_id) AS count
    FROM
       `{}.unioned_ehr_drug_exposure` AS de
    JOIN
      `{}.concept_ancestor` ca
    ON
      de.drug_concept_id = ca.descendant_concept_id
    JOIN
      `{}.concept` c
    ON
      ca.ancestor_concept_id = c.concept_id
    JOIN
      `{}._mapping_drug_exposure`  AS mde
    ON
        de.drug_exposure_id=mde.drug_exposure_id
    WHERE
       c.concept_id IN {}
    AND
        de.drug_concept_id IN {}
    GROUP BY 1
    ORDER BY count DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,top_ccb_ingredient,ccb,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
ccb_ingredient_site.shape

In [ ]:
ccb_ingredient_site

In [ ]:
ccb_ingredient_site['success_rate']=round(100*ccb_ingredient_site['count']/top_option_size,1)
ccb_ingredient_site

In [ ]:
ccb_ingredient_site=ccb_ingredient_site.rename(columns={"success_rate":"ccb"})
ccb_ingredient_site=ccb_ingredient_site[["src_hpo_id","ccb"]]
ccb_ingredient_site

## diuretics ingredient

In [ ]:
# diuretics_ingredient = pd.io.gbq.read_gbq('''
#     SELECT
#         DISTINCT c.concept_id AS ingredient_concept_id, c.concept_name AS ingredient,COUNT(*) AS count
#     FROM
#        `{}.unioned_ehr_drug_exposure` AS de
#     JOIN
#       `{}.concept_ancestor` ca
#     ON
#       de.drug_concept_id = ca.descendant_concept_id
#     JOIN
#       `{}.concept` c
#     ON
#       ca.ancestor_concept_id = c.concept_id
#     WHERE
#       c.concept_class_id = 'Ingredient'
#     AND
#         de.drug_concept_id IN {}
#     GROUP BY 1,2
#     ORDER BY count DESC
#     '''.format(DATASET, DATASET,DATASET,diuretics,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
#                                     dialect='standard'
# )
# diuretics_ingredient.shape

In [ ]:
# diuretics_ingredient

In [ ]:
# top_diuretics_ingredient=tuple(diuretics_ingredient["ingredient_concept_id"].head(20))

In [ ]:
top_diuretics_ingredient=(974166,992590,19010015,948787,904639,
                          19082886,956874,970250,1395058,942350,932745,991382,1309799,904542,978555,907013)

In [ ]:
top_option_size=len(top_diuretics_ingredient)

## diuretics ingredient by sites

In [ ]:
diuretics_ingredient_site = pd.io.gbq.read_gbq('''
    SELECT
        mde.src_hpo_id,
        COUNT(DISTINCT c.concept_id) AS count
    FROM
       `{}.unioned_ehr_drug_exposure` AS de
    JOIN
      `{}.concept_ancestor` ca
    ON
      de.drug_concept_id = ca.descendant_concept_id
    JOIN
      `{}.concept` c
    ON
      ca.ancestor_concept_id = c.concept_id
    JOIN
      `{}._mapping_drug_exposure`  AS mde
    ON
        de.drug_exposure_id=mde.drug_exposure_id
    WHERE
       c.concept_id IN {}
    AND
        de.drug_concept_id IN {}
    GROUP BY 1
    ORDER BY count DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,top_diuretics_ingredient,diuretics,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
diuretics_ingredient_site.shape

In [ ]:
diuretics_ingredient_site

In [ ]:
diuretics_ingredient_site['success_rate']=round(100*diuretics_ingredient_site['count']/top_option_size,1)
diuretics_ingredient_site

In [ ]:
diuretics_ingredient_site=diuretics_ingredient_site.rename(columns={"success_rate":"diuretics"})
diuretics_ingredient_site=diuretics_ingredient_site[["src_hpo_id","diuretics"]]
diuretics_ingredient_site

## opioids ingredient

In [ ]:
# opioids_ingredient = pd.io.gbq.read_gbq('''
#     SELECT
#         DISTINCT c.concept_id AS ingredient_concept_id, c.concept_name AS ingredient,COUNT(*) AS count
#     FROM
#        `{}.unioned_ehr_drug_exposure` AS de
#     JOIN
#       `{}.concept_ancestor` ca
#     ON
#       de.drug_concept_id = ca.descendant_concept_id
#     JOIN
#       `{}.concept` c
#     ON
#       ca.ancestor_concept_id = c.concept_id
#     WHERE
#       c.concept_class_id = 'Ingredient'
#     AND
#         de.drug_concept_id IN {}
#     GROUP BY 1,2
#     ORDER BY count DESC
#     '''.format(DATASET, DATASET,DATASET,opioids,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
#                                     dialect='standard'
# )
# opioids_ingredient.shape

In [ ]:
# opioids_ingredient

In [ ]:
# top_opioids_ingredient=tuple(opioids_ingredient["ingredient_concept_id"].head(20))

In [ ]:
top_opioids_ingredient=(1110410,1124957,1126658,1154029,1103314,1201620,1174888,1102527,1103640)

In [ ]:
top_option_size=len(top_opioids_ingredient)

## opioids ingredient by sites

In [ ]:
opioids_ingredient_site = pd.io.gbq.read_gbq('''
    SELECT
        mde.src_hpo_id,
        COUNT(DISTINCT c.concept_id) AS count
    FROM
       `{}.unioned_ehr_drug_exposure` AS de
    JOIN
      `{}.concept_ancestor` ca
    ON
      de.drug_concept_id = ca.descendant_concept_id
    JOIN
      `{}.concept` c
    ON
      ca.ancestor_concept_id = c.concept_id
    JOIN
      `{}._mapping_drug_exposure`  AS mde
    ON
        de.drug_exposure_id=mde.drug_exposure_id
    WHERE
       c.concept_id IN {}
    AND
        de.drug_concept_id IN {}
    GROUP BY 1
    ORDER BY count DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,top_opioids_ingredient,opioids,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
opioids_ingredient_site.shape

In [ ]:
opioids_ingredient_site

In [ ]:
opioids_ingredient_site['success_rate']=round(100*opioids_ingredient_site['count']/top_option_size,1)
opioids_ingredient_site

In [ ]:
opioids_ingredient_site=opioids_ingredient_site.rename(columns={"success_rate":"opioids"})
opioids_ingredient_site=opioids_ingredient_site[["src_hpo_id","opioids"]]
opioids_ingredient_site

## statins ingredient

In [ ]:
# statins_ingredient = pd.io.gbq.read_gbq('''
#     SELECT
#         DISTINCT c.concept_id AS ingredient_concept_id, c.concept_name AS ingredient,COUNT(*) AS count
#     FROM
#        `{}.unioned_ehr_drug_exposure` AS de
#     JOIN
#       `{}.concept_ancestor` ca
#     ON
#       de.drug_concept_id = ca.descendant_concept_id
#     JOIN
#       `{}.concept` c
#     ON
#       ca.ancestor_concept_id = c.concept_id
#     WHERE
#       c.concept_class_id = 'Ingredient'
#     AND
#         de.drug_concept_id IN {}
#     GROUP BY 1,2
#     ORDER BY count DESC
#     '''.format(DATASET, DATASET,DATASET,statins,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
#                                     dialect='standard'
# )
# antibiotics_ingredient.shape

In [ ]:
# statins_ingredient

In [ ]:
# top_statins_ingredient=tuple(statins_ingredient["ingredient_concept_id"].head(20))

In [ ]:
top_statins_ingredient=(1539403,1592085,1510813,1549686,1545958,40165636,1551860)

In [ ]:
top_option_size=len(top_statins_ingredient)

## statins ingredient by sites

In [ ]:
statins_ingredient_site = pd.io.gbq.read_gbq('''
    SELECT
        mde.src_hpo_id,
        COUNT(DISTINCT c.concept_id) AS count
    FROM
       `{}.unioned_ehr_drug_exposure` AS de
    JOIN
      `{}.concept_ancestor` ca
    ON
      de.drug_concept_id = ca.descendant_concept_id
    JOIN
      `{}.concept` c
    ON
      ca.ancestor_concept_id = c.concept_id
    JOIN
      `{}._mapping_drug_exposure`  AS mde
    ON
        de.drug_exposure_id=mde.drug_exposure_id
    WHERE
       c.concept_id IN {}
    AND
        de.drug_concept_id IN {}
    GROUP BY 1
    ORDER BY count DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,top_statins_ingredient,statins,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
statins_ingredient_site.shape

In [ ]:
statins_ingredient_site

In [ ]:
statins_ingredient_site['success_rate']=round(100*statins_ingredient_site['count']/top_option_size,1)
statins_ingredient_site

In [ ]:
statins_ingredient_site=statins_ingredient_site.rename(columns={"success_rate":"statins"})
statins_ingredient_site=statins_ingredient_site[["src_hpo_id","statins"]]
statins_ingredient_site

## msknaids ingredient

In [ ]:
# msknaids_ingredient = pd.io.gbq.read_gbq('''
#     SELECT
#         DISTINCT c.concept_id AS ingredient_concept_id, c.concept_name AS ingredient,COUNT(*) AS count
#     FROM
#        `{}.unioned_ehr_drug_exposure` AS de
#     JOIN
#       `{}.concept_ancestor` ca
#     ON
#       de.drug_concept_id = ca.descendant_concept_id
#     JOIN
#       `{}.concept` c
#     ON
#       ca.ancestor_concept_id = c.concept_id
#     WHERE
#       c.concept_class_id = 'Ingredient'
#     AND
#         de.drug_concept_id IN {}
#     GROUP BY 1,2
#     ORDER BY count DESC
#     '''.format(DATASET, DATASET,DATASET,msknaids,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
#                                     dialect='standard'
# )
# msknaids_ingredient.shape

In [ ]:
# msknaids_ingredient

In [ ]:
# top_msknaids_ingredient=tuple(msknaids_ingredient["ingredient_concept_id"].head(20))

In [ ]:
top_msknaids_ingredient=(1136980,1115008,1177480,1236607,1150345,1395573,1124300,1146810,1185922,1103374,1178663,1118084)

In [ ]:
top_option_size=len(top_msknaids_ingredient)

## msknaids ingredient by sites

In [ ]:
msknaids_ingredient_site = pd.io.gbq.read_gbq('''
    SELECT
        mde.src_hpo_id,
        COUNT(DISTINCT c.concept_id) AS count
    FROM
       `{}.unioned_ehr_drug_exposure` AS de
    JOIN
      `{}.concept_ancestor` ca
    ON
      de.drug_concept_id = ca.descendant_concept_id
    JOIN
      `{}.concept` c
    ON
      ca.ancestor_concept_id = c.concept_id
    JOIN
      `{}._mapping_drug_exposure`  AS mde
    ON
        de.drug_exposure_id=mde.drug_exposure_id
    WHERE
       c.concept_id IN {}
    AND
        de.drug_concept_id IN {}
    GROUP BY 1
    ORDER BY count DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,top_msknaids_ingredient,msknaids,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
msknaids_ingredient_site.shape

msknaids_ingredient_site

In [ ]:
msknaids_ingredient_site['success_rate']=round(100*msknaids_ingredient_site['count']/top_option_size,1)
msknaids_ingredient_site

In [ ]:
msknaids_ingredient_site=msknaids_ingredient_site.rename(columns={"success_rate":"msknaids"})
msknaids_ingredient_site=msknaids_ingredient_site[["src_hpo_id","msknaids"]]
msknaids_ingredient_site

## oralhypo ingredient

In [ ]:
# oralhypo_ingredient = pd.io.gbq.read_gbq('''
#     SELECT
#         DISTINCT c.concept_id AS ingredient_concept_id, c.concept_name AS ingredient,COUNT(*) AS count
#     FROM
#        `{}.unioned_ehr_drug_exposure` AS de
#     JOIN
#       `{}.concept_ancestor` ca
#     ON
#       de.drug_concept_id = ca.descendant_concept_id
#     JOIN
#       `{}.concept` c
#     ON
#       ca.ancestor_concept_id = c.concept_id
#     WHERE
#       c.concept_class_id = 'Ingredient'
#     AND
#         de.drug_concept_id IN {}
#     GROUP BY 1,2
#     ORDER BY count DESC
#     '''.format(DATASET, DATASET,DATASET,oralhypo,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
#                                     dialect='standard'
# )
# oralhypo_ingredient.shape

In [ ]:
# oralhypo_ingredient

In [ ]:
# top_oralhypo_ingredient=tuple(oralhypo_ingredient["ingredient_concept_id"].head(20))

In [ ]:
top_oralhypo_ingredient=(1503297,1525215,43526465,1560171,1559684,1580747,19097821,45774751,1515249,19059796,1597756,1516766,1502826,1547504,1529331,1510202,43009032,40166035,19122137,40239216,43013884,44785829,43526465,730548)

In [ ]:
top_option_size=len(top_oralhypo_ingredient)

## oralhypo ingredient by sites

In [ ]:
oralhypo_ingredient_site = pd.io.gbq.read_gbq('''
    SELECT
        mde.src_hpo_id,
        COUNT(DISTINCT c.concept_id) AS count
    FROM
       `{}.unioned_ehr_drug_exposure` AS de
    JOIN
      `{}.concept_ancestor` ca
    ON
      de.drug_concept_id = ca.descendant_concept_id
    JOIN
      `{}.concept` c
    ON
      ca.ancestor_concept_id = c.concept_id
    JOIN
      `{}._mapping_drug_exposure`  AS mde
    ON
        de.drug_exposure_id=mde.drug_exposure_id
    WHERE
       c.concept_id IN {}
    AND
        de.drug_concept_id IN {}
    GROUP BY 1
    ORDER BY count DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,top_oralhypo_ingredient,oralhypo,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
oralhypo_ingredient_site.shape

In [ ]:
oralhypo_ingredient_site

In [ ]:
oralhypo_ingredient_site['success_rate']=round(100*oralhypo_ingredient_site['count']/top_option_size,1)
oralhypo_ingredient_site

In [ ]:
oralhypo_ingredient_site=oralhypo_ingredient_site.rename(columns={"success_rate":"oralhypo"})
oralhypo_ingredient_site=oralhypo_ingredient_site[["src_hpo_id","oralhypo"]]
oralhypo_ingredient_site

## painnsaids ingredient

In [ ]:
# painnsaids_ingredient = pd.io.gbq.read_gbq('''
#     SELECT
#         DISTINCT c.concept_id AS ingredient_concept_id, c.concept_name AS ingredient,COUNT(*) AS count
#     FROM
#        `{}.unioned_ehr_drug_exposure` AS de
#     JOIN
#       `{}.concept_ancestor` ca
#     ON
#       de.drug_concept_id = ca.descendant_concept_id
#     JOIN
#       `{}.concept` c
#     ON
#       ca.ancestor_concept_id = c.concept_id
#     WHERE
#       c.concept_class_id = 'Ingredient'
#     AND
#         de.drug_concept_id IN {}
#     GROUP BY 1,2
#     ORDER BY count DESC
#     '''.format(DATASET, DATASET,DATASET,painnsaids,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
#                                     dialect='standard'
# )
# painnsaids_ingredient.shape

In [ ]:
# painnsaids_ingredient

In [ ]:
# top_painnsaids_ingredient=tuple(painnsaids_ingredient["ingredient_concept_id"].head(20))

In [ ]:
top_painnsaids_ingredient=(1125315,1112807,1177480,1185922,1115008)

In [ ]:
top_option_size=len(top_painnsaids_ingredient)

## painnsaids ingredient by sites

In [ ]:
painnsaids_ingredient_site = pd.io.gbq.read_gbq('''
    SELECT
        mde.src_hpo_id,
        COUNT(DISTINCT c.concept_id) AS count
    FROM
       `{}.unioned_ehr_drug_exposure` AS de
    JOIN
      `{}.concept_ancestor` ca
    ON
      de.drug_concept_id = ca.descendant_concept_id
    JOIN
      `{}.concept` c
    ON
      ca.ancestor_concept_id = c.concept_id
    JOIN
      `{}._mapping_drug_exposure`  AS mde
    ON
        de.drug_exposure_id=mde.drug_exposure_id
    WHERE
       c.concept_id IN {}
    AND
        de.drug_concept_id IN {}
    GROUP BY 1
    ORDER BY count DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,top_painnsaids_ingredient,painnsaids,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
painnsaids_ingredient_site.shape

In [ ]:
painnsaids_ingredient_site

In [ ]:
painnsaids_ingredient_site['success_rate']=round(100*painnsaids_ingredient_site['count']/top_option_size,1)
painnsaids_ingredient_site

In [ ]:
painnsaids_ingredient_site=painnsaids_ingredient_site.rename(columns={"success_rate":"painnsaids"})
painnsaids_ingredient_site=painnsaids_ingredient_site[["src_hpo_id","painnsaids"]]
painnsaids_ingredient_site

## vaccine ingredient

In [ ]:
# vaccine_ingredient = pd.io.gbq.read_gbq('''
#     SELECT
#         DISTINCT c.concept_id AS ingredient_concept_id, c.concept_name AS ingredient,COUNT(*) AS count
#     FROM
#        `{}.unioned_ehr_drug_exposure` AS de
#     JOIN
#       `{}.concept_ancestor` ca
#     ON
#       de.drug_concept_id = ca.descendant_concept_id
#     JOIN
#       `{}.concept` c
#     ON
#       ca.ancestor_concept_id = c.concept_id
#     WHERE
#       c.concept_class_id = 'Ingredient'
#     AND
#         de.drug_concept_id IN {}
#     GROUP BY 1,2
#     ORDER BY count DESC
#     '''.format(DATASET, DATASET,DATASET,vaccine,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
#                                     dialect='standard'
# )
# vaccine_ingredient.shape

In [ ]:
# vaccine_ingredient

In [ ]:
# top_vaccine_ingredient=tuple(vaccine_ingredient["ingredient_concept_id"].head(20))

In [ ]:
top_vaccine_ingredient=(42800027,529303, 529411, 529218,596876,528323,19113026,529076,1312375, 1312376,43012953, 
                        43532049, 42903450,509079, 509081, 45775636,40163692,
                        523283, 523365, 523367,529176, 529180, 529212, 529214,792777,19033193,19136047,19045672,46234308)

In [ ]:
top_option_size=len(top_vaccine_ingredient)

## vaccine ingredient by sites

In [ ]:
vaccine_ingredient_site = pd.io.gbq.read_gbq('''
    SELECT
        mde.src_hpo_id,
        COUNT(DISTINCT c.concept_id) AS count
    FROM
       `{}.unioned_ehr_drug_exposure` AS de
    JOIN
      `{}.concept_ancestor` ca
    ON
      de.drug_concept_id = ca.descendant_concept_id
    JOIN
      `{}.concept` c
    ON
      ca.ancestor_concept_id = c.concept_id
    JOIN
      `{}._mapping_drug_exposure`  AS mde
    ON
        de.drug_exposure_id=mde.drug_exposure_id
    WHERE
       c.concept_id IN {}
    AND
        de.drug_concept_id IN {}
    GROUP BY 1
    ORDER BY count DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,top_vaccine_ingredient,vaccine,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
vaccine_ingredient_site.shape

In [ ]:
vaccine_ingredient_site

In [ ]:
vaccine_ingredient_site['success_rate']=round(100*vaccine_ingredient_site['count']/top_option_size,1)
vaccine_ingredient_site

In [ ]:
vaccine_ingredient_site=vaccine_ingredient_site.rename(columns={"success_rate":"vaccine"})
vaccine_ingredient_site=vaccine_ingredient_site[["src_hpo_id","vaccine"]]
vaccine_ingredient_site

## aceinhibitor ingredient

In [ ]:
# aceinhibitor_ingredient = pd.io.gbq.read_gbq('''
#     SELECT
#         DISTINCT c.concept_id AS ingredient_concept_id, c.concept_name AS ingredient,COUNT(*) AS count
#     FROM
#        `{}.unioned_ehr_drug_exposure` AS de
#     JOIN
#       `{}.concept_ancestor` ca
#     ON
#       de.drug_concept_id = ca.descendant_concept_id
#     JOIN
#       `{}.concept` c
#     ON
#       ca.ancestor_concept_id = c.concept_id
#     WHERE
#       c.concept_class_id = 'Ingredient'
#     AND
#         de.drug_concept_id IN {}
#     GROUP BY 1,2
#     ORDER BY count DESC
#     '''.format(DATASET, DATASET,DATASET,aceinhibitor,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
#                                     dialect='standard'
# )
# aceinhibitor_ingredient.shape

In [ ]:
# aceinhibitor_ingredient

In [ ]:
# top_aceinhibitor_ingredient=tuple(aceinhibitor_ingredient["ingredient_concept_id"].head(20))

In [ ]:
top_aceinhibitor_ingredient=(1308216,1341927,1334456,1340128,1335471,1363749,1310756,1373225,1331235,1342439)

In [ ]:
top_option_size=len(top_aceinhibitor_ingredient)

## aceinhibitor ingredient by sites

In [ ]:
aceinhibitor_ingredient_site = pd.io.gbq.read_gbq('''
    SELECT
        mde.src_hpo_id,
        COUNT(DISTINCT c.concept_id) AS count
    FROM
       `{}.unioned_ehr_drug_exposure` AS de
    JOIN
      `{}.concept_ancestor` ca
    ON
      de.drug_concept_id = ca.descendant_concept_id
    JOIN
      `{}.concept` c
    ON
      ca.ancestor_concept_id = c.concept_id
    JOIN
      `{}._mapping_drug_exposure`  AS mde
    ON
        de.drug_exposure_id=mde.drug_exposure_id
    WHERE
       c.concept_id IN {}
    AND
        de.drug_concept_id IN {}
    GROUP BY 1
    ORDER BY count DESC
    '''.format(DATASET, DATASET,DATASET,DATASET,top_aceinhibitor_ingredient,aceinhibitor,DATASET,DATASET,DATASET,drugs,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET,DATASET),
                                    dialect='standard'
)
aceinhibitor_ingredient_site.shape

In [ ]:
aceinhibitor_ingredient_site

In [ ]:
aceinhibitor_ingredient_site['success_rate']=round(100*aceinhibitor_ingredient_site['count']/top_option_size,1)
aceinhibitor_ingredient_site

In [ ]:
aceinhibitor_ingredient_site=aceinhibitor_ingredient_site.rename(columns={"success_rate":"aceinhibitor"})
aceinhibitor_ingredient_site=aceinhibitor_ingredient_site[["src_hpo_id","aceinhibitor"]]
aceinhibitor_ingredient_site

## combined succes rates:4. Integration of Drug Concept Sets:

In [ ]:

succes_rate=pd.merge(aceinhibitor_ingredient_site,vaccine_ingredient_site,how='outer',on='src_hpo_id')
succes_rate=pd.merge(succes_rate,painnsaids_ingredient_site,how='outer',on='src_hpo_id')
succes_rate=pd.merge(succes_rate,oralhypo_ingredient_site,how='outer',on='src_hpo_id')
succes_rate=pd.merge(succes_rate,msknaids_ingredient_site,how='outer',on='src_hpo_id')
succes_rate=pd.merge(succes_rate,statins_ingredient_site,how='outer',on='src_hpo_id')
succes_rate=pd.merge(succes_rate,opioids_ingredient_site,how='outer',on='src_hpo_id')
succes_rate=pd.merge(succes_rate,diuretics_ingredient_site,how='outer',on='src_hpo_id')
#succes_rate=pd.merge(succes_rate,antibiotics_ingredient_site,how='outer',on='src_hpo_id')
succes_rate=pd.merge(succes_rate,ccb_ingredient_site,how='outer',on='src_hpo_id')
succes_rate=pd.merge(succes_rate,site_df,how='outer',on='src_hpo_id')
succes_rate=succes_rate.fillna("No Data")
succes_rate

In [ ]:
succes_rate.to_csv("data\\q4_Integration_of_Drug_Concept_Sets.csv")

# q6. Integration of Measurement Concept Sets:

In [ ]:
cbc=(3015182,3019897,3020416,3023314,3000963,3000905,3012030,3009744,3023599,3002736,3043111,3024929)
cbc_size=len(cbc)

In [ ]:
cbc_diff=(3004809,3022096,3025159,3006504,3009542,3000963,3002030,3035941,3003338,3024731,3002179,3019069,3017181
          ,3018010,3007461,3003214,3021440,3012764,3026361,3027945,3008757,3010813)
cbc_diff_size=len(cbc_diff)

In [ ]:
cmp=(3006923,3024561,3020509,3035995,3013721,3006906,3016723,3045716,3016293,3015632,3014576,3023103,3019550,3027970,3029829,3030354,
     3004501,3020630,3024128,3013682,3018311,3053283,3049187)
cmp_size=len(cmp)    

In [ ]:
bmp=(3006906,3016723,3045716,3016293,3015632,3014576,3023103,3019550,3029829,3030354,3004501,3013682,3018311,3053283,3049187)
bmp_size=len(bmp)  

In [ ]:
phy_mea=(3004249,3012888,3027598,3036277,3019204,3020891,3013762,3027018,3024171)
phy_mea_size=len(phy_mea) 

In [ ]:
lipid=(3019900,3027114,3049873,3007070,3023602,3034482,3053286,3001308,3028288,3028437,3035009,3035899,3038988,42870529,3009966,3053341,3002109,3045001,3007352,3009596,3022487,40768809,3011163,3016087,3007943,3019038,3013678,3022038,3022192,3025839,3027997,42868692)
lipid_size=len(lipid) 

## CBC by sites

In [ ]:
cbc_count = pd.io.gbq.read_gbq('''

            SELECT
                src_hpo_id,
                COUNT(DISTINCT(measurement_concept_id)) AS count
            FROM
               `{}.unioned_ehr_measurement` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS t2
            ON
                t1.measurement_id=t2.measurement_id
            WHERE measurement_concept_id in {}
            GROUP BY
                1
    '''.format(DATASET, DATASET,cbc),
                                    dialect='standard'
)
cbc_count.shape

In [ ]:
cbc_count['cbc']=round(100*cbc_count['count']/cbc_size,1)
cbc_count=cbc_count[["src_hpo_id","cbc"]]
cbc_count

## cbc_diff by sites

In [ ]:
cbc_diff_count = pd.io.gbq.read_gbq('''

            SELECT
                src_hpo_id,
                COUNT(DISTINCT(measurement_concept_id)) AS count
            FROM
               `{}.unioned_ehr_measurement` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS t2
            ON
                t1.measurement_id=t2.measurement_id
            WHERE measurement_concept_id in {}
            GROUP BY
                1
    '''.format(DATASET, DATASET,cbc_diff),
                                    dialect='standard'
)
cbc_diff_count.shape

In [ ]:
cbc_diff_count['cbc_diff']=round(100*cbc_diff_count['count']/cbc_diff_size,1)
cbc_diff_count=cbc_diff_count[["src_hpo_id","cbc_diff"]]
cbc_diff_count

## cmp by sites

In [ ]:
cmp_count = pd.io.gbq.read_gbq('''

            SELECT
                src_hpo_id,
                COUNT(DISTINCT(measurement_concept_id)) AS count
            FROM
               `{}.unioned_ehr_measurement` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS t2
            ON
                t1.measurement_id=t2.measurement_id
            WHERE measurement_concept_id in {}
            GROUP BY
                1
    '''.format(DATASET, DATASET,cmp),
                                    dialect='standard'
)
cmp_count.shape

In [ ]:
cmp_count['cmp']=round(100*cmp_count['count']/cmp_size,1)
cmp_count=cmp_count[["src_hpo_id","cmp"]]
cmp_count

## bmp by sites

In [ ]:
bmp_count = pd.io.gbq.read_gbq('''

            SELECT
                src_hpo_id,
                COUNT(DISTINCT(measurement_concept_id)) AS count
            FROM
               `{}.unioned_ehr_measurement` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS t2
            ON
                t1.measurement_id=t2.measurement_id
            WHERE measurement_concept_id in {}
            GROUP BY
                1
    '''.format(DATASET, DATASET,bmp),
                                    dialect='standard'
)
bmp_count.shape

In [ ]:
bmp_count['bmp']=round(100*bmp_count['count']/bmp_size,1)
bmp_count=bmp_count[["src_hpo_id","bmp"]]
bmp_count

## phy_mea by sites

In [ ]:
phy_mea_count = pd.io.gbq.read_gbq('''

            SELECT
                src_hpo_id,
                COUNT(DISTINCT(measurement_concept_id)) AS count
            FROM
               `{}.unioned_ehr_measurement` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS t2
            ON
                t1.measurement_id=t2.measurement_id
            WHERE measurement_concept_id in {}
            GROUP BY
                1
    '''.format(DATASET, DATASET,phy_mea),
                                    dialect='standard'
)
phy_mea_count.shape

In [ ]:
phy_mea_count['phy_mea']=round(100*phy_mea_count['count']/phy_mea_size,1)
phy_mea_count=phy_mea_count[["src_hpo_id","phy_mea"]]
phy_mea_count

## lipid by sites

In [ ]:
lipid_count = pd.io.gbq.read_gbq('''

            SELECT
                src_hpo_id,
                COUNT(DISTINCT(measurement_concept_id)) AS count
            FROM
               `{}.unioned_ehr_measurement` AS t1
            INNER JOIN
                (SELECT
                    DISTINCT * 
                FROM
                     `{}._mapping_measurement`)  AS t2
            ON
                t1.measurement_id=t2.measurement_id
            WHERE measurement_concept_id in {}
            GROUP BY
                1
    '''.format(DATASET, DATASET,lipid),
                                    dialect='standard'
)
lipid_count.shape

In [ ]:
lipid_count['lipid']=round(100*lipid_count['count']/lipid_size,1)
lipid_count=lipid_count[["src_hpo_id","lipid"]]
lipid_count

## combined succes rates:6. Integration of Measurement Concept Sets:

In [ ]:

succes_rate=pd.merge(lipid_count,phy_mea_count,how='outer',on='src_hpo_id')
succes_rate=pd.merge(succes_rate,bmp_count,how='outer',on='src_hpo_id')
succes_rate=pd.merge(succes_rate,cmp_count,how='outer',on='src_hpo_id')
succes_rate=pd.merge(succes_rate,cbc_diff_count,how='outer',on='src_hpo_id')
succes_rate=pd.merge(succes_rate,cbc_count,how='outer',on='src_hpo_id')
succes_rate=succes_rate.fillna("No Data")
succes_rate

In [ ]:
succes_rate.to_csv("data\\q6_Integration_Measurement_Concept_Sets.csv")